## 代码使用教程-运行环境需求

要成功运行以下代码，需要满足以下技术要求：

### 基本环境
- Python 3.6+
- Firefox浏览器

### Python依赖库
以下库需要安装：
- requests：用于HTTP请求
- beautifulsoup4：用于HTML解析
- selenium：用于浏览器自动化
- dataclasses：用于数据类定义（Python 3.6中需要单独安装，Python 3.7+已内置）

### Selenium相关组件
- geckodriver：Firefox浏览器的WebDriver，用于Selenium驱动Firefox

### 安装说明

1. **安装Firefox浏览器**
   - 从[Firefox官网](https://www.mozilla.org/firefox/new/)下载并安装最新版Firefox

2. **安装geckodriver**
   - 从[geckodriver官方发布页](https://github.com/mozilla/geckodriver/releases)下载适合您操作系统的版本
   - 将geckodriver可执行文件添加到系统PATH环境变量中，或放置在项目目录中

3. **安装Python依赖库**
   ```bash
   pip install requests beautifulsoup4 selenium
   ```

4. **对于Python 3.6，安装dataclasses**
   ```bash
   pip install dataclasses
   ```

### 验证安装
运行以下命令验证核心依赖是否正确安装：
```bash
python -c "import selenium; import bs4; import requests; import dataclasses; print('All dependencies successfully imported.')"
```

如果需要测试Selenium与Firefox的连接，可使用以下简单脚本：
```python
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

options = Options()
service = Service('path/to/geckodriver')  # 如已添加到PATH，可不指定路径
browser = webdriver.Firefox(options=options, service=service)
browser.get('https://www.python.org')
print(f"Title: {browser.title}")
browser.quit()
```

## 数据爬取

In [4]:
import os
import json
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup, Tag
from dataclasses import dataclass, field, asdict
from typing import Dict, List, Any, Set, Optional, Union

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import datetime
import logging

@dataclass
class ModuleContent:
    """统一的模块内容数据类，用于存储和组织不同类型的内容。"""
    content: List[Dict[str, Any]]  # 包含所有类型的完整模块内容
    text: str  # 用于搜索和显示的纯文本内容
    images: List[Dict[str, str]]  # 包含URL和属性的图片信息
    excel_files: List[Dict[str, str]]  # Excel文件链接
    tables: List[List[List[str]]]  # 使用嵌套列表结构的表格数据

@dataclass
class ParameterSpecifications:
    """参数规格数据类，用于存储产品基本规格信息。"""
    product_description: str  # 产品文本描述
    demo_diagram_video: List[Dict[str, str]]  # 产品图表和视频
    general_parameters: Dict[str, Dict[str, str]]  # 产品的一般参数集合

@dataclass
class ModuleProducts:
    """模块内产品集合数据类，用于存储模块内的相关产品信息。"""
    title: str  # 模块标题
    description: str  # 模块描述
    images: List[Dict[str, str]]  # 图片信息列表
    products: List[Dict[str, Any]]  # 产品详情列表

@dataclass
class ProductInfo:
    """产品信息数据类，作为所有产品相关信息的容器"""
    product_name: str  # 产品名称
    image: str  # 产品主图片URL
    category: List[Dict[str, Union[str, int]]]  # 产品分类信息
    parameter_specifications: ParameterSpecifications  # 产品参数规格
    curve_module: Optional[ModuleContent] = None  # 曲线模块（可选）
    technical_details: Optional[ModuleContent] = None  # 技术说明（可选）
    assembly_info: Optional[ModuleContent] = None  # 组装信息（可选）
    application_cases: Optional[ModuleContent] = None  # 应用案例（可选）
    customization_info: Optional[ModuleContent] = None  # 定制能力（可选）
    product_modules: List[ModuleProducts] = field(default_factory=list)  # 产品模块列表

def setup_firefox_driver():
    """设置并返回Firefox浏览器驱动"""
    firefox_options = Options()
    firefox_options.add_argument("--headless")  # 无头模式
    firefox_options.add_argument("--disable-gpu")
    firefox_options.add_argument("--no-sandbox")
    
    driver = webdriver.Firefox(options=firefox_options)
    return driver

class ProductInfoExtractor:
    """产品信息提取器类，负责提取各种类型的产品信息。"""

    def __init__(self, url: str):
        """使用产品页面URL初始化提取器实例。"""
        self.url = url
        self.soup = self._get_soup()
        self.detailed_product_data = []  # 存储通过Selenium获取的详细产品数据

    def _get_soup(self) -> BeautifulSoup:
        """获取页面的BeautifulSoup对象。"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(self.url, headers=headers)
        response.encoding = response.apparent_encoding
        return BeautifulSoup(response.text, 'html.parser')

    def _clean_text(self, text: str) -> str:
        """清理和标准化文本内容。"""
        text = text.replace('\u2003', ' ')
        text = text.replace('^2', '²')
        text = re.sub(r'<a\b[^>]*>(.*?)</a>', r'\1', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def _is_download_button(self, element: Tag) -> bool:
        """检查元素是否为下载按钮。"""
        if element and element.name == 'img':
            return 'download.jpg' in element.get('src', '')
        return False

    def _is_table_file(self, url: str) -> bool:
        """检查URL是否指向Excel文件或CSV文件。"""
        file_name = url.lower()
        return any(ext in file_name for ext in ['xlsx', 'xls', 'csv'])

    def _process_table_row(self, row: Tag, row_index: int = 0) -> Optional[Dict[str, Any]]:
        """处理表格行，并添加位置信息。"""
        cells = row.find_all(['td', 'th'])
        if not cells:
            return None

        if row.find('th'):
            if len(cells) == 1 and 'colspan' in cells[0].attrs:
                text = self._clean_text(cells[0].get_text())
                return {
                    'type': 'header', 
                    'content': text,
                    'position': {'row': row_index, 'colspan': int(cells[0]['colspan'])}
                } if text else None
            else:
                content = []
                positions = []
                for col_idx, cell in enumerate(cells):
                    text = self._clean_text(cell.get_text())
                    content.append(text)
                    colspan = int(cell.get('colspan', 1))
                    positions.append({
                        'col': col_idx, 
                        'colspan': colspan
                    })
                return {
                    'type': 'header', 
                    'content': content,
                    'positions': positions,
                    'row': row_index
                } if any(content) else None

        row_data = []
        positions = []
        current_col = 0

        for cell in cells:
            cell_text = self._clean_text(cell.get_text())
            row_data.append(cell_text)
            colspan = int(cell.get('colspan', 1))
            rowspan = int(cell.get('rowspan', 1))
            positions.append({
                'col': current_col,
                'colspan': colspan,
                'rowspan': rowspan
            })
            current_col += colspan

        return {
            'type': 'data', 
            'content': row_data,
            'positions': positions,
            'row': row_index
        } if any(row_data) else None

    def _process_list(self, ul: Tag, indent_level: int = 0) -> List[Dict[str, Any]]:
        """处理嵌套列表。"""
        list_content = []
        for li in ul.find_all('li', recursive=False):
            item_text = self._clean_text(li.get_text())
            if item_text:
                list_content.append({
                    'type': 'list_item',
                    'content': item_text,
                    'indent': indent_level
                })
            nested_ul = li.find('ul')
            if nested_ul:
                nested_content = self._process_list(nested_ul, indent_level + 1)
                list_content.extend(nested_content)
        return list_content

    def _extract_product_description(self) -> str:
        """提取产品描述文本。"""
        description_parts = []
        explain_div = self.soup.find('div', class_='lb-product-info__explain')

        if explain_div:
            bullet_list = explain_div.find('ul')
            if bullet_list:
                for li in bullet_list.find_all('li'):
                    point_text = self._clean_text(li.text)
                    if point_text:
                        description_parts.append(f"• {point_text}")
                description_parts.append("")

            paragraphs = explain_div.find_all('p')
            if paragraphs:
                for p in paragraphs:
                    para_text = self._clean_text(p.text)
                    if para_text:
                        description_parts.append(para_text)

        return "\n".join(description_parts)

    def _extract_demo_media(self) -> List[Dict[str, str]]:
        """提取产品演示媒体内容。"""
        media = []
        demo_section = self.soup.find('div', id='productSwiper')
        if demo_section:
            for item in demo_section.find_all('div', class_='lb-product-info__swiper_item'):
                media_item = {
                    'type': 'image',
                    'url': item.get('data-img', '') or item.find('img', {'src': True}).get('src', '')
                }
                if media_item['url']:
                    media.append(media_item)
        return media

    def _extract_general_parameters(self) -> Dict[str, Dict[str, str]]:
        """提取产品一般参数。"""
        params = {}
        table_div = self.soup.find('div', class_='lb-product-table')
        if table_div:
            for row in table_div.find_all('tr'):
                for cell in row.find_all('td'):
                    name_div = cell.find('div', class_='t')
                    value_div = cell.find('div', class_='c')
                    if name_div and value_div:
                        name = self._clean_text(name_div.text)
                        value = self._clean_text(value_div.text)
                        if name:
                            params[name] = {
                                'name': name,
                                'value': value
                            }
        return params

    def _extract_category(self, product_url=None) -> List[Dict[str, Union[str, int]]]:
        """提取产品分类信息。"""
        category = []
        nav = self.soup.find('div', class_='lb-breadcrumb')
        if nav:
            spans = nav.find_all('span')
            span_count = len(spans[2:])
            for i, item in enumerate(spans[2:]):
                is_last_item = (i == span_count - 1)
                link = item.find('a')
                if link:
                    href = link.get('href', '')
                    id_match = re.search(r'/product_center/(\d+)\.html', href)
                    id_value = int(id_match.group(1)) if id_match else None
                    category.append({
                        'name': self._clean_text(link.text),
                        'id': id_value
                    })
                else:
                    last_item_text = self._clean_text(item.text)
                    last_item_id = None
                    if is_last_item and product_url:
                        product_id_match = re.search(r'/product/(\d+)', product_url)
                        if product_id_match:
                            last_item_id = int(product_id_match.group(1))
                    category.append({
                        'name': last_item_text,
                        'id': last_item_id
                    })
        return category

    def _extract_module_content(self, module: Tag) -> ModuleContent:
        """提取完整的模块内容，包括文本、图片、表格等。"""
        content = []
        text_parts = []
        images = []
        excel_files = []
        tables = []

        def process_element(element: Tag, current_section: Optional[str] = None) -> None:
            if not isinstance(element, Tag):
                return
            if element.name == 'img' and not self._is_download_button(element):
                img_data = {
                    'type': 'image',
                    'src': element.get('src', ''),
                    'width': element.get('width'),
                    'height': element.get('height')
                }
                images.append(img_data)
                content.append(img_data)
            elif element.name == 'a':
                href = element.get('href', '')
                if href and not href.endswith('download.jpg'):
                    if self._is_table_file(href):
                        file_data = {'type': 'excel', 'url': href}
                        excel_files.append(file_data)
                        content.append(file_data)
            elif element.name == 'p':
                text = self._clean_text(element.get_text())
                if text:
                    style = element.get('style', '')
                    if 'text-indent: 2em' in style:
                        text = f"    {text}"
                    text_data = {'type': 'text', 'content': text}
                    content.append(text_data)
                    text_parts.append(text)
            elif element.name == 'ul':
                list_items = self._process_list(element)
                content.extend(list_items)
                for item in list_items:
                    indent = "    " * item['indent']
                    text_parts.append(f"{indent}• {item['content']}")
            elif element.name == 'table':
                table_data = []
                table_title = None
                caption = element.find('caption')
                if caption:
                    table_title = self._clean_text(caption.get_text())
                for row_idx, row in enumerate(element.find_all('tr')):
                    row_data = self._process_table_row(row, row_idx)
                    if row_data:
                        if not table_title and row_data['type'] == 'header' and isinstance(row_data['content'], str):
                            table_title = row_data['content']
                        table_data.append(row_data)
                if table_data:
                    tables.append(table_data)
                    content.append({'type': 'table', 'data': table_data})
            for child in element.children:
                if isinstance(child, Tag):
                    process_element(child, current_section)

        for element in module.children:
            if isinstance(element, Tag):
                process_element(element)

        return ModuleContent(
            content=content,
            text='\n'.join(text_parts),
            images=images,
            excel_files=excel_files,
            tables=tables
        )

    def _extract_rich_modules(self) -> Dict[str, ModuleContent]:
        """提取所有富文本模块的内容。"""
        modules = {}
        nav_items = self.soup.find_all('div', class_='lb-product-tabs__nav_item')
        content_items = self.soup.find_all('div', class_='lb-product-tabs__content_item')
        module_mapping = {
            '曲线': 'curve_module',
            '技术说明': 'technical_details',
            '组装': 'assembly_info',
            '应用案例': 'application_cases',
            '定制能力': 'customization_info'
        }
        for i, nav in enumerate(nav_items):
            module_name = nav.text.strip()
            if module_name in module_mapping and i < len(content_items):
                modules[module_mapping[module_name]] = self._extract_module_content(content_items[i])
        return modules

    def _process_single_product_row(self, driver, row, max_retries=3, retry_delay=2):
        """处理单个产品行，带有重试机制。"""
        retries = 0
        while retries < max_retries:
            try:
                product_id = row.get_attribute("data-id")
                product_sn = row.get_attribute("data-sn")
                if not product_id or not product_sn:
                    logging.warning("警告: 无法获取产品ID或型号，跳过此行")
                    return None
                expand_button = row.find_element(By.CSS_SELECTOR, "div.lb-goods-table__extend")
                is_open = "is-open" in expand_button.get_attribute("class")
                if not is_open:
                    driver.execute_script("arguments[0].scrollIntoView();", expand_button)
                    time.sleep(0.5)
                    driver.execute_script("arguments[0].click();", expand_button)
                    try:
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.lb-goods-detail"))
                        )
                    except TimeoutException:
                        logging.warning(f"警告: 产品 {product_sn} 的详细信息可能未加载")
                        raise Exception("详细信息加载超时")
                time.sleep(1)
                detail_element = None
                next_row = None
                try:
                    next_row = driver.execute_script("return arguments[0].nextElementSibling;", row)
                except Exception as e:
                    logging.error(f"无法找到下一行: {e}")
                    raise Exception("无法找到详细信息行")
                if next_row:
                    detail_elements = next_row.find_elements(By.CSS_SELECTOR, "div.lb-goods-detail")
                    if detail_elements:
                        detail_element = detail_elements[0]
                if not detail_element:
                    logging.error(f"错误: 无法找到产品 {product_sn} 的详细信息")
                    raise Exception("找不到详细信息元素")
                product_details = {
                    "id": product_id,
                    "model": product_sn
                }
                title_elements = detail_element.find_elements(By.CSS_SELECTOR, "div.lb-goods-detail__title")
                if title_elements:
                    product_details["title"] = title_elements[0].text
                param_rows = detail_element.find_elements(By.CSS_SELECTOR, "div.lb-goods-detail__params")
                for param_row in param_rows:
                    param_spans = param_row.find_elements(By.TAG_NAME, "span")
                    if len(param_spans) >= 2:
                        key = param_spans[0].text.strip()
                        value = param_spans[1].text.strip()
                        product_details[key] = value
                download_files = []
                download_links = detail_element.find_elements(By.CSS_SELECTOR, "a.download-control")
                if download_links:
                    for link in download_links:
                        data_url = link.get_attribute("data-url")
                        data_name = link.get_attribute("data-name")
                        if data_url and data_name:
                            download_files.append({
                                "url": data_url,
                                "name": data_name
                            })
                product_data = {
                    "details": product_details,
                    "download_files": download_files
                }
                return product_data
            except Exception as e:
                retries += 1
                if retries < max_retries:
                    logging.error(f"处理产品行时出错: {e}")
                    logging.info(f"等待 {retry_delay} 秒后进行第 {retries+1}/{max_retries} 次重试...")
                    time.sleep(retry_delay)
                else:
                    logging.error(f"达到重试上限 ({max_retries}次)，无法获取 {product_sn} (ID: {product_id}) 的产品信息: {e}")
                    return None

    def _scrape_detailed_product_info(self, max_retries=3, retry_delay=2):
        """使用Selenium抓取产品详细信息。"""
        driver = setup_firefox_driver()
        try:
            driver.get(self.url)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.lb-goods-table__extend"))
                )
            except TimeoutException:
                logging.warning("警告: 页面未加载完成或找不到展开按钮")
                self.detailed_product_data = []
                return
            product_rows = driver.find_elements(By.CSS_SELECTOR, "tr[name='goods']")
            logging.info(f"找到 {len(product_rows)} 个产品行")
            all_product_data = []
            for idx, row in enumerate(product_rows):
                product_data = self._process_single_product_row(driver, row, max_retries, retry_delay)
                if product_data:
                    all_product_data.append(product_data)
                else:
                    logging.info(f"跳过第 {idx+1} 个产品，继续处理下一个")
            self.detailed_product_data = all_product_data
        except Exception as e:
            logging.error(f"爬取过程中出现错误: {e}")
            self.detailed_product_data = []
        finally:
            driver.quit()

    def _get_table_headers(self, table: BeautifulSoup) -> List[Dict[str, int]]:
        """从表格中获取所有列头及其位置。"""
        headers = []
        header_row = table.select_one('thead tr')
        if header_row:
            for idx, th in enumerate(header_row.find_all('th')):
                text = th.get_text().strip()
                if text and text not in ['对比', '']:
                    headers.append({'name': text, 'index': idx})
        return headers

    def _extract_product_data(self, row: BeautifulSoup, headers: List[Dict[str, int]]) -> Dict[str, str]:
        """从表格行中提取产品数据。"""
        product_data = {}
        cells = row.find_all('td')
        for header in headers:
            header_name = header['name']
            cell_idx = header['index']
            if cell_idx < len(cells):
                cell = cells[cell_idx]
                if header_name == '单价':
                    price_elem = cell.select_one('.lb-goods-table__price')
                    value = price_elem.text.strip() if price_elem else ''
                elif header_name == '发货日期':
                    delivery_elem = cell.select_one('span')
                    value = delivery_elem.text.strip() if delivery_elem else ''
                else:
                    value = cell.text.strip()
                product_data[header_name] = value
        return product_data

    def _extract_module_products(self, module: BeautifulSoup) -> ModuleProducts:
        """从产品模块中提取信息。"""
        title_elem = module.select_one('.lb-product-set__title span')
        title = title_elem.text.strip() if title_elem else ''
        description_parts = []
        features = module.select('.lb-product-set__info_explain ul li')
        if features:
            for feature in features:
                feature_text = self._clean_text(feature.text)
                if feature_text:
                    description_parts.append(f"• {feature_text}")
        description = module.select_one('.lb-product-set__info_explain p')
        if description:
            if description_parts:
                description_parts.append('')
            description_text = self._clean_text(description.text)
            if description_text:
                description_parts.append(description_text)
        images = []
        annotations = set()
        for img_div in module.select('.lb-product-set__info_img'):
            img_url = img_div.get('data-img', '')
            if img_url:
                images.append({'url': img_url})
                annotation_elem = img_div.find_next(class_='lb-product-set__info_annotate')
                if annotation_elem:
                    annotation = self._clean_text(annotation_elem.text)
                    if annotation and annotation not in annotations:
                        annotations.add(annotation)
                        description_parts.append('')
                        description_parts.append(f"图片注释：{annotation}")
        products = []
        table = module.select_one('.lb-goods-table')
        if table:
            headers = self._get_table_headers(table)
            for row in table.select('tr[name="goods"]'):
                model = row.get('data-sn', '')
                product_data = self._extract_product_data(row, headers)
                if product_data:
                    if model:
                        product_data['model'] = model
                    products.append(product_data)
        return ModuleProducts(
            title=title,
            description='\n'.join(description_parts),
            images=images,
            products=products
        )

    def _merge_detailed_info_with_products(self, product_modules):
        """将详细产品信息与产品模块信息合并。"""
        if not self.detailed_product_data:
            return product_modules
        model_to_details = {}
        for product_data in self.detailed_product_data:
            details = product_data.get('details', {})
            model = details.get('model')
            if model:
                model_to_details[model] = product_data
        for module in product_modules:
            updated_products = []
            for product in module.products:
                model = product.get('model')
                new_product = {
                    "basic": {k: v for k, v in product.items() if k != 'model'}
                }
                if model and model in model_to_details:
                    detailed_data = model_to_details[model]
                    if 'details' in detailed_data:
                        new_product['details'] = detailed_data['details']
                    if 'download_files' in detailed_data and detailed_data['download_files']:
                        new_product['download_files'] = detailed_data['download_files']
                updated_products.append(new_product)
            module.products = updated_products
        return product_modules
    
    def extract_product_info(self, scrape_details=True, max_retries=3, retry_delay=2) -> ProductInfo:
        """提取所有产品信息并处理。"""
        try:
            parameter_specs = ParameterSpecifications(
                product_description=self._extract_product_description(),
                demo_diagram_video=self._extract_demo_media(),
                general_parameters=self._extract_general_parameters()
            )
            product_modules = []
            for module in self.soup.select('.lb-product-set'):
                module_info = self._extract_module_products(module)
                if module_info.products:
                    product_modules.append(module_info)
            rich_modules = self._extract_rich_modules()
            if scrape_details:
                logging.info("开始抓取详细产品信息...")
                self._scrape_detailed_product_info(max_retries, retry_delay)
                product_modules = self._merge_detailed_info_with_products(product_modules)
            product_title_elem = self.soup.find('div', class_='lb-product__banner_title')
            product_title = product_title_elem.text.strip() if product_title_elem else ""
            product_img_elem = self.soup.find('div', class_='lb-product__banner_img')
            product_img = product_img_elem.find('img').get('src', '') if product_img_elem else ""
            product_info = ProductInfo(
                product_name=product_title,
                image=product_img,
                category=self._extract_category(self.url),
                parameter_specifications=parameter_specs,
                **rich_modules,
                product_modules=product_modules,
            )
            return product_info
        except Exception as e:
            logging.error(f"提取产品信息时发生错误: {str(e)}")
            raise

def sanitize_filename(filename: str) -> str:
    """净化文件名，仅替换操作系统不允许使用的特殊字符。"""
    invalid_chars = r'[/\\:*?"<>|]'
    return re.sub(invalid_chars, '_', filename)

def process_product_url(url, scrape_details=True, max_retries=3, retry_delay=2):
    """处理单个产品URL。"""
    try:
        logging.info(f"开始处理URL: {url}")
        extractor = ProductInfoExtractor(url)
        product_info = extractor.extract_product_info(scrape_details, max_retries, retry_delay)
        product_dict = asdict(product_info)
        json_output = json.dumps(product_dict, ensure_ascii=False, indent=2)
        return (product_info.product_name, json_output)
    except Exception as e:
        logging.error(f"处理URL时出错: {str(e)}")
        return (None, None)

# 目标文件夹名称
log_folder_name = "日志"

# 从当前目录及其子目录中查找名为“日志”的文件夹
def find_log_folder(start_path="."):
    for root, dirs, _ in os.walk(start_path):
        if log_folder_name in dirs:
            return os.path.join(root, log_folder_name)
    return None  # 如果未找到，返回 None

# 查找日志文件夹
log_folder = find_log_folder()

# 如果未找到，则在当前目录创建
if log_folder is None:
    log_folder = os.path.join(os.getcwd(), log_folder_name)
    os.makedirs(log_folder, exist_ok=True)

# 生成日志文件名
log_file_name = "爬取日志.txt"
log_file_path = os.path.join(log_folder, log_file_name)

# 配置日志，将所有信息写入文件；控制台只通过 log_and_print 输出关键信息
logging.shutdown()
logging.basicConfig(
    filename=log_file_path,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w',
)

# 清除已有的日志处理器，避免重复输出
logger = logging.getLogger()
if logger.hasHandlers():
    logger.handlers.clear()

logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_file_path, mode='w', encoding='utf-8-sig')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

def log_and_print(message):
    """同时打印到控制台并写入日志文件，用于关键信息。"""
    print(message)
    logging.info(message)

def main():
    """主程序入口函数"""
    start_time = time.time()
    try:
        csv_filename = "D:\\Code\\Python_Code\\Jupyter_Notebook\\e-commerce project\\product_links.csv" #请改为你的文件路径
        df = pd.read_csv(csv_filename)
        if 'Address' not in df.columns:
            raise ValueError("CSV 文件中缺少 'Address' 列")
        #设置处理链接数量（例如批量处理还是全部处理）
        urls = df['Address'].dropna().tolist()[:3]
        scrape_details = True
        max_retries = 100
        retry_delay = 1
        results = []
        for i, url in enumerate(urls, 1):
            log_and_print(f"\n正在处理第 {i}/{len(urls)} 个URL: {url}")
            result = process_product_url(url, scrape_details, max_retries, retry_delay)
            if result[0] is not None:
                product_name, json_output = result
                output_folder = "爬取结果"
                os.makedirs(output_folder, exist_ok=True)
                filename = f"{sanitize_filename(product_name)}.json"
                output_path = os.path.join(output_folder, filename)
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(json_output)
                log_and_print(f"已保存文件到: {output_path}")
                results.append({
                    'url': url,
                    'status': 'success',
                    'file': filename
                })
            else:
                log_and_print(f"处理URL失败: {url}")
                results.append({
                    'url': url,
                    'status': 'error'
                })
        log_and_print("\n处理摘要:")
        successful = sum(1 for r in results if r['status'] == 'success')
        failed = sum(1 for r in results if r['status'] == 'error')
        log_and_print(f"处理的URL总数: {len(urls)}")
        log_and_print(f"成功: {successful}")
        log_and_print(f"失败: {failed}")
    except Exception as e:
        log_and_print(f"\n程序执行错误: {str(e)}")
    finally:
        end_time = time.time()
        elapsed_time = end_time - start_time
        log_and_print(f"\n总执行时间: {elapsed_time:.2f} 秒")
        

if __name__ == "__main__":
    main()


正在处理第 1/3 个URL: http://www.lbtek.com/product/1001?fid=36&sid=78&tid=112
已保存文件到: 爬取结果\光栅调整架转接件.json

正在处理第 2/3 个URL: http://www.lbtek.com/product/1002?fid=129&sid=172&tid=
已保存文件到: 爬取结果\三轴光纤准直器.json

正在处理第 3/3 个URL: http://www.lbtek.com/product/1003?fid=129&sid=172&tid=
已保存文件到: 爬取结果\带单模尾纤非球面光纤准直器.json

处理摘要:
处理的URL总数: 3
成功: 3
失败: 0

总执行时间: 498.49 秒


## 数据分割和整理
#### 直接覆盖同名文件，不再进行同名处理

In [1]:
import json
import csv
import os
import re
import datetime
import shutil
from pathlib import Path

def sanitize_filename(filename: str) -> str:
    """净化文件名，仅替换操作系统不允许使用的特殊字符
    
    Args:
        filename: 原始文件名
        
    Returns:
        处理后的安全文件名
    """
    # 定义文件系统不允许的字符
    invalid_chars = r'[/\\:*?"<>|]'
    # 只替换这些特定的非法字符为下划线
    return re.sub(invalid_chars, '_', filename)

# 不再需要此函数，因为我们将直接覆盖文件
# def get_unique_filename(filepath):
#     """处理文件名冲突，返回唯一的文件路径"""
#     ...

def create_directory_structure(base_output_dir):
    """创建标准的输出目录结构"""
    # 创建CSV目录
    csv_dir = os.path.join(base_output_dir, "CSV")
    os.makedirs(csv_dir, exist_ok=True)
    
    # 创建落地页目录
    landing_page_dir = os.path.join(base_output_dir, "落地页")
    os.makedirs(landing_page_dir, exist_ok=True)
    
    # 创建产品详情主目录
    product_details_dir = os.path.join(base_output_dir, "产品详情")
    os.makedirs(product_details_dir, exist_ok=True)
    
    return {
        "csv_dir": csv_dir,
        "landing_page_dir": landing_page_dir,
        "product_details_dir": product_details_dir
    }

def get_current_timestamp():
    """获取当前日期时间字符串"""
    return datetime.datetime.now().strftime("%Y%m%d")

def process_json_file(input_file_path, output_dir):
    """处理单个JSON文件"""
    # 创建日志字符串，记录处理过程
    log_entries = [f"开始处理文件: {input_file_path}"]
    print(f"开始处理文件: {input_file_path}")
    
    # 读取JSON文件
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 创建目录结构
    dirs = create_directory_structure(output_dir)
    
    # 获取安全的文件名基础部分（不含扩展名）
    file_name = os.path.basename(input_file_path)
    file_name_without_ext = os.path.splitext(file_name)[0]
    safe_file_name = sanitize_filename(file_name_without_ext)
    
    # 1. 删除定制能力text中的指定部分文字
    if isinstance(data.get("customization_info"), dict) and "text" in data["customization_info"]:
        text = data["customization_info"]["text"]
        pattern = r"有/无；.*?(?=若您需要的参数不在上表覆盖范围内)"
        data["customization_info"]["text"] = re.sub(pattern, "", text, flags=re.DOTALL)
    
    # 2. 创建产品系列专用目录（用于产品详情）
    product_series_dir = os.path.join(dirs["product_details_dir"], safe_file_name)
    os.makedirs(product_series_dir, exist_ok=True)
    log_entries.append(f"创建产品系列目录: {product_series_dir}")
    
    # 3. 创建CSV产品列表
    csv_result = create_product_csv(data, safe_file_name, dirs["csv_dir"])
    log_entries.append(csv_result)
    
    # 4. 创建落地页描述文件，并获取内容
    landing_page_file, landing_page_content = create_landing_page_description(data, safe_file_name, dirs["landing_page_dir"])
    log_entries.append(f"创建落地页描述文件: {landing_page_file}")
    
    # 5. 为每个产品创建详细信息文件，并获取内容字典
    product_details_files, product_details_contents = create_product_details_files(data, safe_file_name, product_series_dir)
    log_entries.extend([f"创建产品详情文件: {f}" for f in product_details_files])
    
    # 6. 创建通用内容文件
    common_content_file = create_common_content_file(data, safe_file_name, product_series_dir)
    log_entries.append(f"创建通用内容文件: {common_content_file}")
    
    # 返回日志条目
    return log_entries

def create_product_csv(data, file_name, csv_dir):
    """创建CSV产品列表文件"""
    csv_file_name = os.path.join(csv_dir, f"{file_name}_产品列表.csv")
    
    # 不再检查文件是否存在，直接使用文件名（将覆盖已有文件）
    # if os.path.exists(csv_file_name):
    #     timestamp = get_current_timestamp()
    #     csv_file_name = os.path.join(csv_dir, f"{file_name}_产品列表_{timestamp}.csv")
    
    # 新格式的列名
    fixed_columns = ["商品名称", "品牌", "商品编码"]
    
    # 收集所有可能的字段名
    basic_fields = set()
    details_fields = set()
    
    # 检查数据结构中的产品模块
    products = []
    for module in data.get("product_modules", []) or []:
        for product in module.get("products", []) or []:
            # 创建新的产品字典
            combined_product = {}
            
            # 添加固定字段
            combined_product["商品名称"] = data.get("product_name", "")
            combined_product["品牌"] = "Labtek"
            combined_product["商品编码"] = ""
            
            # 收集并添加所有字段
            # 兼容原脚本中的"basic"字段，也支持直接的键值结构
            if isinstance(product, dict):
                product_data = {}
                
                # 处理basic字段情况
                if "basic" in product and isinstance(product["basic"], dict):
                    for key, value in product["basic"].items():
                        if key != "单价":
                            if key == "发货日期":
                                # 重命名发货日期为货期，并替换值
                                key = "货期"
                                if value == "当天":
                                    value = "现货"
                                else:
                                    value = "期货"
                            basic_fields.add(key)
                            product_data[key] = value
                    
                    # 提取价格（如果有）
                    if "单价" in product["basic"]:
                        price_str = product["basic"]["单价"]
                        # 去掉货币符号，只保留数值
                        price = re.sub(r'[^\d]', '', price_str)
                        product_data["人民币（可更改）"] = price
                
                # 处理details字段情况
                if "details" in product and isinstance(product["details"], dict):
                    for key, value in product["details"].items():
                        details_fields.add(key)
                        product_data[key] = value
                
                # 处理直接键值结构（如"产品型号"等直接在product中的字段）
                for key, value in product.items():
                    if key not in ["basic", "details"]:
                        # 对于直接字段，判断是否是基本字段
                        if key in ["产品型号", "阶数m", "工作波长λ", "通光孔径", "发货日期"]:
                            if key == "发货日期":
                                # 重命名发货日期为货期，并替换值
                                key = "货期"
                                if value == "当天":
                                    value = "现货"
                                else:
                                    value = "期货"
                            basic_fields.add(key)
                        else:
                            details_fields.add(key)
                        product_data[key] = value
                
                # 如果有"单价"直接在product中
                if "单价" in product and "人民币（可更改）" not in product_data:
                    price_str = product["单价"]
                    price = re.sub(r'[^\d]', '', price_str)
                    product_data["人民币（可更改）"] = price
                
                # 更新combined_product
                combined_product.update(product_data)
            
            # 添加空的库存和外币字段（如果不存在）
            if "库存（可更改）" not in combined_product:
                combined_product["库存（可更改）"] = ""
            if "外币（可更改）" not in combined_product:
                combined_product["外币（可更改）"] = ""
            if "人民币（可更改）" not in combined_product:
                combined_product["人民币（可更改）"] = ""
            
            products.append(combined_product)
    
    # 组合所有列，按照指定顺序
    columns = fixed_columns + sorted(list(basic_fields)) + ["库存（可更改）", "外币（可更改）", "人民币（可更改）"] + sorted(list(details_fields))
    
    # 写入CSV文件，添加标题行
    with open(csv_file_name, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=columns)
        writer.writeheader()
        for product in products:
            # 确保所有列都有值（即使是空值）
            row = {col: product.get(col, "") for col in columns}
            writer.writerow(row)
    
    return f"创建CSV文件: {csv_file_name}，包含{len(products)}个产品"

def create_landing_page_description(data, file_name, landing_page_dir):
    """创建落地页描述文件，并返回文件路径和内容"""
    txt_file_name = os.path.join(landing_page_dir, f"{file_name}_落地页描述.txt")
    
    # 不再检查文件是否存在，直接使用文件名（将覆盖已有文件）
    # if os.path.exists(txt_file_name):
    #     timestamp = get_current_timestamp()
    #     txt_file_name = os.path.join(landing_page_dir, f"{file_name}_落地页描述_{timestamp}.txt")
    
    content = ""
    # 创建内存中的内容字符串
    # 添加文件头部信息
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
    content += f"# {file_name} - 落地页描述\n"
    content += f"生成时间：{timestamp}\n"
    content += f"来源文件：{file_name}.json\n\n"
    content += "-" * 30 + "\n\n"
    
    # 添加product_name
    if "product_name" in data:
        content += f"## 产品名称\n{data['product_name']}\n\n"
    
    # 添加image
    if "image" in data:
        content += f"## 主图\n{data['image']}\n\n"
    
    # 添加category
    if isinstance(data.get("category"), list):
        content += "## 类别\n"
        for category in data["category"]:
            if isinstance(category, dict):
                content += f"- {category.get('name', '')} (ID: {category.get('id', '')})\n"
        content += "\n"
    
    # 添加parameter_specifications（不包含general_parameters）
    if isinstance(data.get("parameter_specifications"), dict):
        specs = data["parameter_specifications"]
        
        if "product_description" in specs:
            content += "## 产品描述\n"
            content += specs["product_description"]
            content += "\n\n"
        
        if isinstance(specs.get("demo_diagram_video"), list):
            content += "## 演示图/视频\n"
            for item in specs["demo_diagram_video"]:
                if isinstance(item, dict):
                    item_type = item.get("type", "")
                    item_url = item.get("url", "")
                    content += f"- {item_type}: {item_url}\n"
    
    # 写入文件
    with open(txt_file_name, 'w', encoding='utf-8') as f:
        f.write(content)
    
    return txt_file_name, content

def handle_table_data(table_data):
    """安全处理表格数据"""
    result = []
    
    if not table_data:
        return result
        
    for row in table_data:
        row_content = []
        
        # 处理不同类型的行数据
        if isinstance(row, str):
            row_content.append(row)
        elif isinstance(row, list):
            for cell in row:
                if isinstance(cell, str):
                    row_content.append(cell)
                elif isinstance(cell, dict):
                    if 'content' in cell:
                        content = cell['content']
                        if isinstance(content, list):
                            row_content.append(' '.join([str(item) for item in content]))
                        else:
                            row_content.append(str(content))
                    else:
                        row_content.append("")
        elif isinstance(row, dict):
            if 'content' in row:
                content = row['content']
                if isinstance(content, list):
                    row_content.append(' '.join([str(item) for item in content]))
                else:
                    row_content.append(str(content))
        
        if row_content:
            result.append(' | '.join(row_content))
    
    return result

def create_product_details_files(data, file_name, product_series_dir):
    """为每个产品创建详细信息文件，仅包含产品相关信息"""
    file_paths = []
    content_dict = {}
    
    product_count = 0
    unknown_count = 0
    
    # 为每个产品创建详细信息文件
    for module in data.get("product_modules", []) or []:
        module_title = module.get("title", "")
        module_description = module.get("description", "")
        module_images = module.get("images", []) or []
        
        for product in module.get("products", []) or []:
            product_count += 1
            
            # 确定产品型号
            model = ""
            
            # 尝试从details中获取
            if isinstance(product.get("details"), dict):
                details = product["details"]
                if "model" in details:
                    model = details["model"]
                elif "产品型号" in details:
                    model = details["产品型号"]
            
            # 如果details中没有，尝试直接从product中获取
            if not model and "产品型号" in product:
                model = product["产品型号"]
            
            # 如果仍然没有找到，使用unknown加数字
            if not model:
                unknown_count += 1
                model = f"unknown_{unknown_count:03d}"
            
            # 安全处理模型名称
            safe_model = sanitize_filename(model)
            
            # 创建文件路径
            txt_file_name = os.path.join(product_series_dir, f"{file_name}_{safe_model}.txt")
            
            # 不再检查文件名冲突，直接覆盖
            # if os.path.exists(txt_file_name):
            #     txt_file_name = get_unique_filename(txt_file_name)
            
            # 提取产品详情
            product_details = {}
            if isinstance(product.get("details"), dict):
                product_details = product["details"]
            else:
                # 收集直接字段（排除special_fields）
                special_fields = ["basic", "download_files"]
                for key, value in product.items():
                    if key not in special_fields and not isinstance(value, (dict, list)):
                        product_details[key] = value
            
            # 创建内存中的内容字符串
            content = ""
            
            # 添加文件头部信息
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
            content += f"# {file_name} - {safe_model}\n"
            content += f"生成时间：{timestamp}\n"
            content += f"来源文件：{file_name}.json\n\n"
            content += "-" * 30 + "\n\n"
            
            # 添加标题和描述
            content += f"## {module_title}\n\n"
            content += f"{module_description}\n\n"
            
            # 添加模块图片
            if module_images:
                content += "### 模块图片\n"
                for img in module_images:
                    if isinstance(img, dict):
                        content += f"![图片]({img.get('url', '')})\n"
                content += "\n"
            
            # 添加产品详情
            content += "## 产品详情\n"
            for key, value in product_details.items():
                content += f"- {key}: {value}\n"
            content += "\n"
            
            # 添加下载文件
            if isinstance(product.get("download_files"), list):
                content += "### 下载文件\n"
                for file_info in product["download_files"]:
                    if isinstance(file_info, dict):
                        content += f"- [{file_info.get('name', '')}]({file_info.get('url', '')})\n"
                content += "\n"
            
            # 写入文件
            with open(txt_file_name, 'w', encoding='utf-8') as f:
                f.write(content)
            
            # 保存文件路径和内容
            file_paths.append(txt_file_name)
            content_dict[safe_model] = content
    
    return file_paths, content_dict

def create_common_content_file(data, file_name, product_series_dir):
    """创建包含通用内容的文件"""
    # 创建文件路径
    txt_file_name = os.path.join(product_series_dir, f"{file_name}_通用内容.txt")
    
    # 不再检查文件名冲突，直接覆盖
    # if os.path.exists(txt_file_name):
    #     timestamp = get_current_timestamp()
    #     txt_file_name = os.path.join(product_series_dir, f"{file_name}_通用内容_{timestamp}.txt")
    
    # 创建内存中的内容字符串
    content = ""
    
    # 添加文件头部信息
    #timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
    #content += f"# {file_name} - 通用内容\n"
    #content += f"生成时间：{timestamp}\n"
    #content += f"来源文件：{file_name}.json\n\n"
    #content += "-" * 30 + "\n\n"
    
    # 添加曲线模块
    if isinstance(data.get("curve_module"), dict):
        content += "## 曲线模块\n\n"
        curve_module = data["curve_module"]
        
        if curve_module.get("text", ""):
            content += f"{curve_module['text']}\n\n"
        
        if isinstance(curve_module.get("images"), list) and curve_module["images"]:
            content += "### 曲线图片\n"
            for img in curve_module["images"]:
                if isinstance(img, dict):
                    content += f"![曲线图]({img.get('src', '')})\n"
            content += "\n"
        
        if isinstance(curve_module.get("excel_files"), list) and curve_module["excel_files"]:
            content += "### 曲线Excel文件\n"
            for excel in curve_module["excel_files"]:
                if isinstance(excel, dict):
                    content += f"- [{excel.get('type', '')}]({excel.get('url', '')})\n"
            content += "\n"
    
    # 添加技术细节
    if isinstance(data.get("technical_details"), dict):
        content += "## 技术细节\n\n"
        tech_details = data["technical_details"]
        
        if tech_details.get("text", ""):
            content += f"{tech_details['text']}\n\n"
            
        if isinstance(tech_details.get("images"), list) and tech_details["images"]:
            content += "### 技术图片\n"
            for img in tech_details["images"]:
                if isinstance(img, dict):
                    content += f"![技术图]({img.get('src', '')})\n"
            content += "\n"
    
    # 添加装配信息
    if isinstance(data.get("assembly_info"), dict):
        content += "## 装配信息\n\n"
        assembly = data["assembly_info"]
        
        if assembly.get("text", ""):
            content += f"{assembly['text']}\n\n"
        
        if isinstance(assembly.get("images"), list) and assembly["images"]:
            content += "### 装配图片\n"
            for img in assembly["images"]:
                if isinstance(img, dict):
                    content += f"![装配图]({img.get('src', '')})\n"
            content += "\n"
        
        if isinstance(assembly.get("tables"), list) and assembly["tables"]:
            content += "### 装配表格\n"
            table_rows = handle_table_data(assembly["tables"])
            for row in table_rows:
                content += f"- {row}\n"
            content += "\n"
    
    # 添加应用案例
    if isinstance(data.get("application_cases"), dict):
        content += "## 应用案例\n\n"
        cases = data["application_cases"]
        
        if cases.get("text", ""):
            content += f"{cases['text']}\n\n"
        
        if isinstance(cases.get("images"), list) and cases["images"]:
            content += "### 应用案例图片\n"
            for img in cases["images"]:
                if isinstance(img, dict):
                    content += f"![应用案例]({img.get('src', '')})\n"
            content += "\n"
    
    # 添加定制信息
    if isinstance(data.get("customization_info"), dict):
        content += "## 定制信息\n\n"
        customization = data["customization_info"]
        
        if customization.get("text", ""):
            content += f"{customization['text']}\n\n"
        
        # 直接以JSON格式输出定制表格，保持原始格式
        if isinstance(customization.get("tables"), list) and customization["tables"]:
            content += "### 定制表格 (原始格式)\n"
            content += "```json\n"
            json_str = json.dumps(customization["tables"], ensure_ascii=False, indent=2)
            content += json_str
            content += "\n```\n\n"
    
    # 写入文件
    with open(txt_file_name, 'w', encoding='utf-8') as f:
        f.write(content)
    
    return txt_file_name

def process_all_json_files(input_dir, output_dir):
    """处理输入目录中的所有JSON文件"""
    # 首先确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建日志文件夹（与输入文件夹平行）
    log_dir = os.path.join(os.path.dirname(input_dir), "日志")
    os.makedirs(log_dir, exist_ok=True)
    
    # 创建日志文件
    log_file_path = os.path.join(log_dir, f"拆分日志_{get_current_timestamp()}.txt")
    
    # 确保输入目录存在
    if not os.path.exists(input_dir):
        with open(log_file_path, 'w', encoding='utf-8') as log_file:
            log_file.write(f"错误: 输入目录不存在: {input_dir}\n")
        print(f"输入目录不存在: {input_dir}")
        return
    
    # 获取输入目录下的所有JSON文件
    json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    
    if not json_files:
        with open(log_file_path, 'w', encoding='utf-8') as log_file:
            log_file.write(f"警告: 在 {input_dir} 目录下没有找到JSON文件\n")
        print(f"在 {input_dir} 目录下没有找到JSON文件")
        return
    
    # 开始处理文件并记录日志
    with open(log_file_path, 'w', encoding='utf-8') as log_file:
        log_file.write(f"处理开始时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        log_file.write(f"输入目录: {input_dir}\n")
        log_file.write(f"输出目录: {output_dir}\n")
        log_file.write(f"日志目录: {log_dir}\n")
        log_file.write(f"找到 {len(json_files)} 个JSON文件待处理\n\n")
        
        for json_file in json_files:
            input_file_path = os.path.join(input_dir, json_file)
            log_file.write(f"\n{'=' * 50}\n")
            log_file.write(f"开始处理文件: {input_file_path}\n")
            
            try:
                log_entries = process_json_file(input_file_path, output_dir)
                for entry in log_entries:
                    log_file.write(f"{entry}\n")
                log_file.write(f"文件处理完成: {json_file}\n")
            except Exception as e:
                log_file.write(f"处理文件 {json_file} 时出错: {str(e)}\n")
                import traceback
                log_file.write(traceback.format_exc())
        
        log_file.write(f"\n处理结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    
    print(f"所有文件处理完毕! 详细日志请查看: {log_file_path}")

if __name__ == "__main__":
    input_directory = r'./爬取结果'  # 修改为你的输入文件夹路径
    output_directory = r'./拆分结果'  # 修改为你的输出文件夹路径
    
    print(f"开始处理，输入目录: {input_directory}，输出目录: {output_directory}")
    process_all_json_files(input_directory, output_directory)

开始处理，输入目录: ./爬取结果，输出目录: ./拆分结果
开始处理文件: ./爬取结果\25 mm行程手动线性位移台.json
开始处理文件: ./爬取结果\30 mm同轴分束立方安装座.json
开始处理文件: ./爬取结果\30 mm同轴安装板.json
开始处理文件: ./爬取结果\30 mm同轴平片光学元件安装座.json
开始处理文件: ./爬取结果\30 mm同轴立方体.json
开始处理文件: ./爬取结果\30 mm同轴立方体配件.json
开始处理文件: ./爬取结果\30 mm同轴系统45度安装调整架.json
开始处理文件: ./爬取结果\30 mm同轴系统XYZ位移调整架.json
开始处理文件: ./爬取结果\30 mm同轴系统XY位移调整架.json
开始处理文件: ./爬取结果\30 mm同轴系统反射镜架.json
开始处理文件: ./爬取结果\30 mm同轴系统对准板.json
开始处理文件: ./爬取结果\30 mm同轴系统旋转调整架.json
开始处理文件: ./爬取结果\45度安装调整架.json
开始处理文件: ./爬取结果\60 mm同轴安装板.json
开始处理文件: ./爬取结果\60 mm同轴系统反射镜架.json
开始处理文件: ./爬取结果\60 mm同轴系统旋转调整架.json
开始处理文件: ./爬取结果\OAS光学分析软件.json
开始处理文件: ./爬取结果\SM05透镜套筒.json
开始处理文件: ./爬取结果\SM1透镜套筒.json
开始处理文件: ./爬取结果\SM2透镜套筒.json
开始处理文件: ./爬取结果\Ø12.7 mm不锈钢光学接杆.json
开始处理文件: ./爬取结果\Ø12.7 mm接杆套环.json
开始处理文件: ./爬取结果\Ø12.7 mm接杆支架.json
开始处理文件: ./爬取结果\Ø12.7 mm接杆角度夹具.json
开始处理文件: ./爬取结果\Ø6 mm同轴接杆转接件.json
开始处理文件: ./爬取结果\万向调节镜架.json
开始处理文件: ./爬取结果\三轴光纤准直器.json
开始处理文件: ./爬取结果\二维调整架，表面中心线和接杆安装孔中心对齐.json
开始处理文件: ./爬取结果\便携式数字光功率计.json
开始处理文件: ./爬

## 集成翻译、SEO优化和质量反馈系统

In [ ]:

# -*- coding: utf-8 -*-
"""
集成翻译、SEO优化和质量反馈系统
创建于 Sun Mar 16 13:12:05 2025

功能：
- 精准翻译技术内容（特别是光学术语）
- 自动提取SEO关键词
- 翻译质量评估
- 批量处理文件和目录
- 支持CSV文件翻译
- 智能拆分超长文件
"""

import os
import json
import time
import sys
import requests
from pathlib import Path
import re
import logging
import datetime
import argparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
import math
import io
import csv


def setup_logging(log_level=logging.INFO):
    """设置日志系统，使用UTF-8编码避免Windows环境下的编码错误"""
    log_format = "%(asctime)s - %(levelname)s - %(message)s"
    log_dir = "日志"
    os.makedirs(log_dir, exist_ok=True)  # 确保日志文件夹存在
    log_filename = os.path.join(log_dir, "翻译日志.txt")
    
    # 配置控制台输出处理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(log_level)
    console_handler.setFormatter(logging.Formatter(log_format))
    
    # 配置文件输出处理器，明确指定编码为utf-8
    file_handler = logging.FileHandler(log_filename, encoding='utf-8')
    file_handler.setLevel(log_level)
    file_handler.setFormatter(logging.Formatter(log_format))
    
    # 设置根日志记录器
    logger = logging.getLogger(__name__)
    logger.setLevel(log_level)
    
    # 清除可能存在的处理器
    logger.handlers = []
    
    # 添加新的处理器
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    return logger

# 使用单例模式确保logger只被初始化一次
_logger_instance = None

def get_logger():
    global _logger_instance
    if _logger_instance is None:
        _logger_instance = setup_logging()
    return _logger_instance

# 在模块级别初始化logger
logger = get_logger()

class SimpleOpenAIClient:
    """简单的OpenAI API客户端，只使用基本库"""

    def __init__(self, api_key, api_base=None):
        self.api_key = api_key
        self.api_base = api_base or "https://api.openai.com/v1"
        self.base_url = f"{self.api_base}/chat/completions"

    def create_chat_completion(self, model, messages, temperature=0.3, max_tokens=None):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature
        }

        # 如果指定了最大令牌数，添加到请求中
        if max_tokens:
            payload["max_tokens"] = max_tokens

        try:
            response = requests.post(
                self.base_url,
                headers=headers,
                json=payload,
                timeout=180  # 增加到180秒，解决超时问题
            )

            if response.status_code != 200:
                error_msg = f"API错误: {response.status_code} - {response.text}"
                logger.error(error_msg)
                raise Exception(error_msg)

            return response.json()

        except requests.exceptions.Timeout:
            raise Exception("API请求超时，请稍后重试")
        except requests.exceptions.ConnectionError:
            raise Exception("连接错误，请检查网络连接")
        except Exception as e:
            raise Exception(f"API请求失败: {str(e)}")


def count_tokens(text):
    """估计文本中的token数量，用于判断是否需要拆分"""
    # 简单估计：英文约每4个字符1个token，中文约每1.5个字符1个token
    en_char_count = len(re.findall(r'[a-zA-Z0-9]', text))
    zh_char_count = len(re.findall(r'[\u4e00-\u9fff]', text))
    other_char_count = len(text) - en_char_count - zh_char_count

    token_estimate = (en_char_count / 4) + (zh_char_count / 1.5) + (other_char_count / 3)
    return math.ceil(token_estimate)



def split_csv_improved(csv_content, max_rows=5):
    """将CSV文件内容拆分为多个小块，每块固定最多max_rows行，并确保保留表头"""
    try:
        # 使用csv模块正确解析CSV内容，处理包含逗号的字段
        reader = csv.reader(io.StringIO(csv_content))
        header = next(reader)  # 获取表头
        rows = list(reader)
        
        chunks = []
        num_chunks = math.ceil(len(rows) / max_rows)
        
        for i in range(num_chunks):
            start_idx = i * max_rows
            end_idx = min((i + 1) * max_rows, len(rows))
            
            chunk_rows = rows[start_idx:end_idx]
            
            # 使用csv模块正确生成CSV内容，自动处理特殊字符
            output = io.StringIO()
            writer = csv.writer(output, quoting=csv.QUOTE_ALL)  # 使用QUOTE_ALL确保所有字段被引号包围
            writer.writerow(header)
            writer.writerows(chunk_rows)
            
            chunks.append(output.getvalue())
            output.close()
        
        logger.info(f"将CSV拆分为{len(chunks)}个块，每块最多{max_rows}行")
        return chunks, header
    except Exception as e:
        logger.error(f"CSV拆分错误: {e}")
        return [], None


def extract_seo_keywords(api_key, model, content, keyword_count=10, api_base=None):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """从内容中提取SEO关键词"""
    logger.info("开始提取SEO关键词...")

    # 构建系统提示
    system_prompt = f"""你是一名专业的SEO优化专家，负责提取高相关关键词。

    任务目标：
    1. 基于原文自动提取高相关关键词
    2. 确保关键词符合SEO规则（搜索量高，竞争适中）
    3. 生成主关键词（Primary Keywords）+ 相关长尾关键词（Long-Tail Keywords）
    4. 确保关键词与内容一致，避免无关词汇

    请按以下JSON格式返回{keyword_count}个关键词，按搜索流量潜力排序：
    ```json
    {{
        "primary_keywords": ["关键词1", "关键词2", "关键词3"],
        "long_tail_keywords": ["长尾关键词1", "长尾关键词2", "..."]
    }}
    ```
    """

    # 创建客户端
    client = SimpleOpenAIClient(api_key, api_base)

    # API调用及错误处理和重试
    max_retries = 3
    retry_delay = 2

    for attempt in range(max_retries):
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": content}
            ]

            response = client.create_chat_completion(model, messages, temperature=0.3)
            result = response["choices"][0]["message"]["content"]

            # 尝试解析JSON结果
            try:
                # 查找JSON格式的内容
                json_match = re.search(r'```json\s*([\s\S]*?)\s*```|(\{[\s\S]*\})', result)
                if json_match:
                    json_str = json_match.group(1) or json_match.group(2)
                    keywords = json.loads(json_str)
                    logger.info(
                        f"成功提取关键词：{len(keywords.get('primary_keywords', [])) + len(keywords.get('long_tail_keywords', []))}个")
                    return keywords
                else:
                    # 如果没有JSON格式，尝试直接提取关键词列表
                    keywords_list = []
                    for line in result.split('\n'):
                        if '-' in line or ':' in line:
                            keyword = line.split('-')[-1].split(':')[-1].strip()
                            if keyword and not keyword.startswith(('```', '{')):
                                keywords_list.append(keyword)

                    if keywords_list:
                        logger.info(f"成功提取关键词（非标准格式）：{len(keywords_list)}个")
                        return {"keywords": keywords_list}
                    else:
                        # 如果还是没有提取到，返回原始结果
                        logger.warning("无法解析关键词结果，返回原始文本")
                        return {"raw_result": result}
            except json.JSONDecodeError:
                # 如果解析JSON失败，返回文本格式
                logger.warning("JSON解析失败，返回原始文本")
                return {"raw_result": result}

        except Exception as e:
            if "rate_limit" in str(e).lower() and attempt < max_retries - 1:
                logger.warning(f"遇到速率限制，{retry_delay}秒后重试...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 指数退避
            else:
                logger.error(f"提取关键词出错: {str(e)}")
                raise


def generate_ai_feedback(api_key, model, original_content, translated_content, api_base=None):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """生成AI翻译质量反馈"""
    logger.info("开始生成翻译质量评估...")

    # 构建系统提示
    system_prompt = """你是一位专业的翻译质量评估专家。
    评估以下翻译的质量，关注以下几点：
    1. 术语准确性：特别是专业术语（如光学术语）的翻译是否准确
    2. 语法和流畅度：翻译是否自然流畅
    3. 格式保留：是否保留了原文的格式和结构
    4. 内容完整性：是否有遗漏或添加内容

    请按以下JSON格式提供评估：
    ```json
    {
        "overall_score": 4.5,  // 总体评分 (1-5分，5分为最高)
        "terminology_score": 4.0,  // 术语准确性评分
        "fluency_score": 4.5,  // 流畅度评分
        "formatting_score": 5.0,  // 格式保留评分
        "completeness_score": 4.8,  // 完整性评分
        "strengths": ["优点1", "优点2", "..."],  // 翻译的优点
        "improvement_areas": ["改进建议1", "改进建议2", "..."],  // 改进建议
        "critical_errors": ["错误1", "错误2", "..."]  // 严重错误（如有）
    }
    ```
    """

    # 创建客户端
    client = SimpleOpenAIClient(api_key, api_base)

    # API调用及错误处理和重试
    max_retries = 3
    retry_delay = 2

    for attempt in range(max_retries):
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"原文:\n\n{original_content}\n\n翻译:\n\n{translated_content}"}
            ]

            response = client.create_chat_completion(model, messages, temperature=0.3)
            result = response["choices"][0]["message"]["content"]

            # 尝试解析JSON结果
            try:
                # 查找JSON格式的内容
                json_match = re.search(r'```json\s*([\s\S]*?)\s*```|(\{[\s\S]*\})', result)
                if json_match:
                    json_str = json_match.group(1) or json_match.group(2)
                    # 移除可能存在的JSON注释
                    json_str = re.sub(r'//.*?$', '', json_str, flags=re.MULTILINE)
                    feedback = json.loads(json_str)
                    logger.info(f"成功生成翻译质量评估，总体评分：{feedback.get('overall_score', 'N/A')}")
                    return feedback
                else:
                    # 如果没有JSON格式，返回原始结果
                    logger.warning("无法解析反馈结果，返回原始文本")
                    return {"raw_feedback": result}
            except json.JSONDecodeError:
                # 如果解析JSON失败，返回文本格式
                logger.warning("JSON解析失败，返回原始文本")
                return {"raw_feedback": result}

        except Exception as e:
            if "rate_limit" in str(e).lower() and attempt < max_retries - 1:
                logger.warning(f"遇到速率限制，{retry_delay}秒后重试...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 指数退避
            else:
                logger.error(f"生成翻译质量反馈出错: {str(e)}")
                raise

def filter_general_content(product_text, general_text):
    # 更智能地处理标题和段落
    def normalize_text(text):
        # 去除多余空白，保留基本格式
        return re.sub(r'\s+', ' ', text).strip()

    paragraphs = [p.strip() for p in re.split(r'\n\n+', general_text) if p.strip()]
    missing_paragraphs = []

    for para in paragraphs:
        # 检查是否为标题
        if re.match(r'^#+\s', para):
            # 标题始终保留
            missing_paragraphs.append(para)
        else:
            # 普通段落检查是否已存在
            normalized_para = normalize_text(para)
            product_normalized = normalize_text(product_text)
            
            if normalized_para not in product_normalized:
                missing_paragraphs.append(para)

    return '\n\n'.join(missing_paragraphs)

def translate_and_optimize(api_key, model, content, target_language, seo_keywords=None,
                           preserve_formatting=True, custom_instructions="", api_base=None):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """使用AI API翻译和优化内容，可选择包含SEO关键词优化"""
    logger.info(f"开始翻译内容到{target_language}...")
    
    # 估算token数量
    estimated_tokens = count_tokens(content)
    logger.info(f"估计内容有 {estimated_tokens} tokens")
    
    # 如果内容超过3000 tokens，直接拒绝处理，要求外部分块
    if estimated_tokens > 3000:
        logger.warning(f"内容长度为 {estimated_tokens} tokens，超过安全限制，需要拆分")
        raise ValueError("Content too long for model context")
    
    # 根据内容长度动态调整完成tokens
    completion_tokens = min(4096, 6000 - estimated_tokens)
    if completion_tokens < 1000:
        completion_tokens = 1000  # 确保至少有1000个token用于回复
        
    logger.info(f"设置完成tokens: {completion_tokens}")
    
    # 构建系统提示，增强技术文档翻译能力
    system_prompt = f"""你是一位专业的技术文档翻译专家，特别擅长光学、光电子领域的精准翻译。

翻译要求：
1. 术语翻译准则：
   - 光学专业术语必须准确翻译，优先使用国际通用术语
   - 对于专业名词，首次出现时可添加英文原文
   - 保持行业术语的一致性和专业性

2. 格式保留要求：
   - 严格保留原始Markdown格式，包括：
     * 标题层级（### 二级标题、#### 三级标题）
     * 列表缩进和类型
     * 代码块
     * 行内格式（粗体、斜体、链接）
   - 标题翻译时保持原始 # 符号数量
   - 代码块和技术描述保持原始排版

3. 语言风格：
   - 使用专业、精确的技术语言
   - 保持原文的技术深度和专业性
   - 确保译文清晰、简洁，便于理解

4. 特殊处理：
   - 数学公式、代码段、技术参数原样保留
   - 对于不确定的专业术语，可在译文中保留原文

翻译目标语言：{target_language}"""
    
    # SEO关键词处理
    keywords_to_use = []
    if seo_keywords:
        # 从多种输入形式提取关键词
        if isinstance(seo_keywords, list):
            keywords_to_use = seo_keywords[:5]
        elif isinstance(seo_keywords, dict):
            keywords_to_use = (
                seo_keywords.get("keywords", []) or 
                seo_keywords.get("primary_keywords", []) + 
                seo_keywords.get("long_tail_keywords", [])
            )[:5]
        
        # 将关键词转换为实际可用的列表
        keywords_to_use = [str(kw).strip() for kw in keywords_to_use if kw]
    
    # 如果有SEO关键词，添加优化指导
    if keywords_to_use:
        keywords_str = ", ".join(keywords_to_use)
        system_prompt += f"""

SEO优化指南：
- 自然融入关键词：{keywords_str}
- 关键词密度控制在1-2%
- 保持内容流畅性和专业性
- 避免过度使用关键词"""
        
        logger.info(f"使用SEO关键词优化：{keywords_str}")
    
    # 添加自定义指令
    if custom_instructions:
        system_prompt += f"\n\n自定义翻译指令：{custom_instructions}"
    
    # 创建客户端
    client = SimpleOpenAIClient(api_key, api_base)
    
    # API调用重试机制
    max_retries = 3
    retry_delay = 2
    
    for attempt in range(max_retries):
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": content}
            ]
            
            # 调用API翻译
            response = client.create_chat_completion(
                model, 
                messages, 
                temperature=0.3, 
                max_tokens=completion_tokens
            )
            
            translated_content = response["choices"][0]["message"]["content"].strip()
            logger.info("翻译完成。")
            return translated_content
        
        except Exception as e:
            # 错误处理逻辑
            if "context_length_exceeded" in str(e):
                logger.warning("内容过长，需要拆分处理")
                raise ValueError("Content too long for model context")
            elif "rate_limit" in str(e).lower() and attempt < max_retries - 1:
                logger.warning(f"遇到速率限制，{retry_delay}秒后重试...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 指数退避
            else:
                logger.error(f"API 请求失败，重试 {attempt + 1}/{max_retries}: {e}")
                time.sleep(retry_delay)
    
    logger.error("API 请求失败，达到最大重试次数。")
    return None


def split_text_balanced(text, max_tokens=3000):
    """更平衡的文本分块策略，尽量保持语义单元和JSON结构的完整性，并增强标题和格式保留"""
    # 检测是否包含JSON结构
    has_json = bool(re.search(r'{\s*"[^"]+"\s*:', text)) or bool(re.search(r'\[\s*{\s*"[^"]+"\s*:', text))
    
    # 对于包含JSON的文本，使用更大的块大小
    if has_json:
        logger.info("检测到JSON结构，增大块大小")
        max_tokens = max(max_tokens, 4000)  # 增大JSON文本的块大小
    
    # 增强的标题和格式检测正则
    def is_significant_boundary(line):
        # 检测标题、分隔线、代码块开始/结束等
        return (re.match(r'^\s*#{1,6}\s+', line) or  # Markdown标题
                re.match(r'^\s*[-=]{3,}\s*$', line) or  # 分隔线
                re.match(r'^\s*```', line) or  # 代码块开始/结束
                re.match(r'^\s*\*{3,}\s*$', line))  # 星号分隔线
    
    # 按自然分界点分块，保留更多结构信息
    lines = text.split('\n')
    natural_chunks = []
    current_chunk = []
    current_headers = []  # 跟踪当前块的标题层级
    
    for line in lines:
        current_chunk.append(line)
        
        # 检测标题和重要分界点
        if is_significant_boundary(line):
            # 如果是标题，记录其层级
            if re.match(r'^\s*#{1,6}\s+', line):
                header_level = len(re.match(r'^\s*(#+)', line).group(1))
                current_headers.append(header_level)
            
            # 如果块过大，强制切分
            if count_tokens('\n'.join(current_chunk)) > max_tokens:
                natural_chunks.append('\n'.join(current_chunk))
                current_chunk = [line]
                current_headers = [header_level] if 'header_level' in locals() else []
    
    # 添加最后一个块
    if current_chunk:
        natural_chunks.append('\n'.join(current_chunk))
    
    result_chunks = []
    for chunk in natural_chunks:
        chunk_tokens = count_tokens(chunk)
        
        # JSON结构特殊处理
        if has_json and re.search(r'{\s*"[^"]+"\s*:', chunk):
            try:
                # 更复杂的JSON结构检测和保护
                json_matches = list(re.finditer(r'(\{[\s\S]*?}|\[[\s\S]*?])', chunk))
                if json_matches:
                    # 保留完整的JSON结构和上下文
                    processed_chunk = chunk
                    for match in json_matches:
                        json_part = match.group(0)
                        # 确保JSON结构完整
                        processed_chunk = processed_chunk.replace(json_part, f"【JSON_BLOCK_{len(result_chunks)}】")
                    
                    # 分割处理，保留上下文
                    sub_chunks = []
                    current_sub_chunk = []
                    current_sub_tokens = 0
                    
                    for line in processed_chunk.split('\n'):
                        line_tokens = count_tokens(line)
                        
                        if current_sub_tokens + line_tokens > max_tokens:
                            sub_chunks.append('\n'.join(current_sub_chunk))
                            current_sub_chunk = [line]
                            current_sub_tokens = line_tokens
                        else:
                            current_sub_chunk.append(line)
                            current_sub_tokens += line_tokens
                    
                    if current_sub_chunk:
                        sub_chunks.append('\n'.join(current_sub_chunk))
                    
                    # 还原JSON块
                    final_sub_chunks = []
                    for sub_chunk in sub_chunks:
                        for i, match in enumerate(json_matches):
                            placeholder = f"【JSON_BLOCK_{i}】"
                            if placeholder in sub_chunk:
                                sub_chunk = sub_chunk.replace(placeholder, match.group(0))
                        final_sub_chunks.append(sub_chunk)
                    
                    result_chunks.extend(final_sub_chunks)
                    continue
            except Exception as e:
                logger.warning(f"JSON处理复杂逻辑失败: {e}")
        
        # 普通文本处理
        if chunk_tokens <= max_tokens:
            result_chunks.append(chunk)
        else:
            # 按段落再次分割
            paragraphs = re.split(r'\n\s*\n', chunk)
            current_para_chunk = []
            current_para_tokens = 0
            
            for para in paragraphs:
                para_tokens = count_tokens(para)
                
                if current_para_tokens + para_tokens > max_tokens:
                    result_chunks.append('\n\n'.join(current_para_chunk))
                    current_para_chunk = [para]
                    current_para_tokens = para_tokens
                else:
                    current_para_chunk.append(para)
                    current_para_tokens += para_tokens
            
            if current_para_chunk:
                result_chunks.append('\n\n'.join(current_para_chunk))
    
    # 最终合并小块
    final_chunks = []
    current_chunk = ""
    current_tokens = 0
    
    for chunk in result_chunks:
        chunk_tokens = count_tokens(chunk)
        
        if not current_chunk or current_tokens + chunk_tokens <= max_tokens:
            current_chunk = current_chunk + ("\n\n" + chunk if current_chunk else chunk)
            current_tokens += chunk_tokens
        else:
            final_chunks.append(current_chunk)
            current_chunk = chunk
            current_tokens = chunk_tokens
    
    if current_chunk:
        final_chunks.append(current_chunk)
    
    logger.info(f"文本拆分为{len(final_chunks)}个块，最大块大小为{max_tokens} tokens")
    return final_chunks

def process_text_file(api_key, model, file_path, output_path, target_language, seo_optimize, preserve_formatting,
                      custom_instructions, api_base=None):
    """处理文本文件，支持智能拆分和合并，使用更平衡的切割策略"""
    # 添加延迟以避免API速率限制
    time.sleep(2)
    
    logger.info(f"处理文本文件: {file_path}")

    try:
        # 读取文件内容
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='gb18030') as file:
                content = file.read()
        except Exception as e:
            logger.error(f"无法读取文件 {file_path}: {e}")
            return False

    try:
        # 提取SEO关键词（如果启用）
        keywords = None
        if seo_optimize:
            try:
                keywords = extract_seo_keywords(api_key, model, content, keyword_count=5, api_base=api_base)
                logger.info(f"成功提取关键词: {keywords}")
            except Exception as e:
                logger.warning(f"提取关键词出错: {e}")
                keywords = None

        # 尝试直接翻译整个文件
        try:
            # 估算token数量
            estimated_tokens = count_tokens(content)
            logger.info(f"文件总内容估计有 {estimated_tokens} tokens")
            
            # 对于小文件，尝试直接翻译
            if estimated_tokens <= 3000:
                logger.info("文件较小，尝试直接翻译")
                translated_content = translate_and_optimize(
                    api_key, model, content, target_language, keywords,
                    preserve_formatting, custom_instructions, api_base
                )

                # 保存翻译结果
                with open(output_path, 'w', encoding='utf-8') as file:
                    file.write(translated_content)

                logger.info(f"文件翻译完成: {output_path}")
                return True
            else:
                logger.info("文件较大，直接进行拆分处理")
                # 继续执行下面的拆分逻辑
                raise ValueError("Content too long for model context")
                
        except ValueError as e:
            if "Content too long" in str(e):
                logger.info("文件内容过长，进行拆分处理")
                # 继续执行下面的拆分逻辑
            else:
                raise
        except Exception as e:
            logger.error(f"直接翻译失败: {e}")
            # 尝试拆分处理作为备选方案
            logger.info("尝试拆分处理...")

        # 使用更平衡的拆分策略
        chunks = split_text_balanced(content, max_tokens=3000)
        translated_chunks = []

        logger.info(f"文本拆分为{len(chunks)}个块进行处理")

        # 跟踪SEO优化是否已应用
        seo_applied = False

        # 翻译每个块
        for i, chunk in enumerate(chunks):
            logger.info(f"处理文本块 {i + 1}/{len(chunks)}")
            
            # 确定是否应用SEO优化到当前块
            should_apply_seo = seo_optimize and not seo_applied and len(chunk) > 100 and re.search(r'[a-zA-Z\u4e00-\u9fff]', chunk)
            chunk_keywords = keywords if should_apply_seo else None
            
            # 直接翻译块
            success = False
            max_retries = 3  # 减少重试次数，但增加每次等待时间
            retry_delay = 15
            
            for attempt in range(max_retries):
                try:
                    # 添加额外延迟
                    time.sleep(3)
                    
                    translated_chunk = translate_and_optimize(
                        api_key, model, chunk, target_language,
                        chunk_keywords, preserve_formatting, custom_instructions, api_base
                    )
                    
                    if translated_chunk:  # 确保结果不为None
                        translated_chunks.append(translated_chunk)
                        # 如果应用了SEO优化，标记为已应用
                        if should_apply_seo:
                            seo_applied = True
                        success = True
                        break
                except Exception as chunk_e:
                    if "rate_limit" in str(chunk_e).lower() and attempt < max_retries - 1:
                        wait_time = retry_delay * (2 ** attempt)
                        logger.warning(f"API速率限制，等待{wait_time}秒后重试...")
                        time.sleep(wait_time)
                    elif "Content too long" in str(chunk_e) and attempt < max_retries - 1:
                        # 如果块太大，尝试进一步拆分
                        logger.warning(f"块 {i + 1} 太大，进一步拆分处理")
                        sub_chunks = split_text_balanced(chunk, max_tokens=1500)
                        sub_translated = []
                        
                        for j, sub_chunk in enumerate(sub_chunks):
                            logger.info(f"处理子块 {j + 1}/{len(sub_chunks)}")
                            # 添加额外延迟
                            time.sleep(5)
                            
                            sub_should_apply_seo = should_apply_seo and not seo_applied and j == 0
                            sub_keywords = keywords if sub_should_apply_seo else None
                            
                            sub_result = translate_and_optimize(
                                api_key, model, sub_chunk, target_language,
                                sub_keywords, preserve_formatting, custom_instructions, api_base
                            )
                            
                            if sub_result:
                                sub_translated.append(sub_result)
                                if sub_should_apply_seo:
                                    seo_applied = True
                        
                        if sub_translated:
                            # 合并子块
                            translated_chunks.extend(sub_translated)
                            success = True
                            break
                    else:
                        logger.error(f"处理文本块 {i + 1} 失败: {chunk_e}")
                        if attempt == max_retries - 1:
                            # 在最后一次尝试失败后添加错误占位符
                            translated_chunks.append(f"[翻译错误: 块 {i+1}]")
            
            if not success and not translated_chunks[-1].startswith("[翻译错误:"):
                # 如果没有成功也没有添加错误标记，则添加错误标记
                translated_chunks.append(f"[翻译错误: 块 {i+1}]")
            
            # 每个块处理后增加延迟以避免速率限制
            time.sleep(10)  # 较长的延迟

        # 合并翻译结果前处理所有块
        valid_chunks = []
        for chunk in translated_chunks:
            if chunk is not None:
                # 跳过翻译错误标记
                if chunk.startswith("[翻译错误:") and chunk.endswith("]"):
                    logger.info(f"跳过错误标记: '{chunk}'")
                    continue
                
                # 处理后的块内容
                processed_chunk = chunk
                
                # 过滤掉常见的提示语（只有当整个块就是提示语时）
                unwanted_prompts = [
                    "请输入翻译的文本",
                    "Please enter the text to translate",
                    "Please input text for translation",
                    "Please enter text to translate",
                    "Please provide text to translate"
                ]
                
                # 如果整个块就是提示语，则跳过
                if any(processed_chunk.strip() == prompt for prompt in unwanted_prompts):
                    logger.info("已过滤掉纯提示语块")
                    continue
                
                # 检查是否为纯符号或空白块
                if re.match(r'^[\s\W]*$', processed_chunk):
                    logger.info("已过滤掉只包含符号或空白的块")
                    continue
                
                # 处理包含解释性注释的块
                explanation_patterns = [
                    r"The text provided seems to be",
                    r"This appears to be",
                    r"Please note that the translation",
                    r"I've translated the",
                    r"I have translated",
                    r"I can translate",
                    r"I notice this is",
                    r"However, please note that",
                    r"The content appears to be",
                    r"This text contains",
                    r"This is a technical"
                ]
                
                # 检测块是否包含解释性注释
                contains_explanation = False
                for pattern in explanation_patterns:
                    if re.search(pattern, processed_chunk):
                        contains_explanation = True
                        break
                
                # 如果包含解释性注释，尝试提取有效内容
                if contains_explanation:
                    # 拆分块为段落
                    paragraphs = re.split(r'\n\n+', processed_chunk)
                    
                    # 过滤掉包含解释性文本的段落
                    filtered_paragraphs = []
                    for para in paragraphs:
                        is_explanation = False
                        for pattern in explanation_patterns:
                            if re.search(pattern, para):
                                is_explanation = True
                                logger.info(f"过滤掉解释性段落: '{para[:50]}...'")
                                break
                        
                        # 检查是否为纯粹的提示语
                        is_prompt = False
                        for prompt in unwanted_prompts:
                            if para.strip() == prompt:
                                is_prompt = True
                                break
                        
                        if not is_explanation and not is_prompt and para.strip():
                            filtered_paragraphs.append(para)
                    
                    # 重组过滤后的段落
                    if filtered_paragraphs:
                        processed_chunk = '\n\n'.join(filtered_paragraphs)
                        logger.info("已过滤解释性文本，保留有效内容")
                    else:
                        # 如果没有有效段落，则跳过此块
                        logger.info("块中没有有效内容，整块跳过")
                        continue
                
                # 移除 <userStyle> 标记但保留其他内容
                if "<userStyle>" in processed_chunk:
                    processed_chunk = re.sub(r'<userStyle>.*?</userStyle>', '', processed_chunk)
                    logger.info("已移除 <userStyle> 标记，保留其他内容")
                
                # 如果处理后的块不为空，添加到有效块列表
                if processed_chunk.strip():
                    valid_chunks.append(processed_chunk)
        
        if not valid_chunks:
            logger.error("所有文本块翻译失败或为空")
            return False
            
        # 合并翻译结果
        merged_content = '\n\n'.join(valid_chunks)

        # 在这里添加标题修复（对合并后的内容）
        # 注意：你需要定义restore_section_headers函数来实现标题恢复
        merged_content = restore_section_headers(content, merged_content)

        # 最终检查：移除末尾的提示语
        for prompt in unwanted_prompts:
            if merged_content.endswith(prompt):
                merged_content = merged_content[:-len(prompt)].rstrip()
                logger.info(f"已从翻译结果末尾移除提示语: '{prompt}'")

        # 最终检查：移除任何残留的 <userStyle> 标记
        merged_content = re.sub(r'<userStyle>.*?</userStyle>', '', merged_content)

        # 保存合并后的翻译结果 (只需要一次写入操作)
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(merged_content)

        logger.info(f"文本拆分翻译合并完成: {output_path}")
        return True

    except Exception as e:
        logger.error(f"处理文本文件错误: {e}")
        import traceback
        logger.error(traceback.format_exc())  # 打印详细堆栈跟踪
        return False


def split_text(text, max_tokens=1000):
    """按段落拆分文本为多个小块，每块不超过max_tokens，针对超长文本优化"""
    # 按段落拆分
    paragraphs = re.split(r'\n\s*\n', text)
    chunks = []
    current_chunk = []
    current_token_count = 0

    for para in paragraphs:
        para_token_count = count_tokens(para)

        # 如果单个段落超过最大token限制，需要进一步拆分
        if para_token_count > max_tokens:
            # 如果当前chunk不为空，先保存当前chunk
            if current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = []
                current_token_count = 0
                
            # 按句子拆分段落
            sentences = re.split(r'(?<=[.!?。！？])\s*', para)
            
            # 进一步拆分特别长的句子
            processed_sentences = []
            for sentence in sentences:
                sentence_token_count = count_tokens(sentence)
                
                if sentence_token_count > max_tokens:
                    # 如果单个句子仍然超长，按照固定字符数拆分
                    char_chunks = [sentence[i:i+300] for i in range(0, len(sentence), 300)]
                    processed_sentences.extend(char_chunks)
                else:
                    processed_sentences.append(sentence)
            
            # 处理拆分后的句子
            sentence_chunk = []
            sentence_chunk_tokens = 0
            
            for sentence in processed_sentences:
                sentence_token_count = count_tokens(sentence)
                
                # 如果当前句子束加上新句子会超限，保存当前束并开始新束
                if sentence_chunk_tokens + sentence_token_count > max_tokens and sentence_chunk:
                    chunks.append(' '.join(sentence_chunk))
                    sentence_chunk = []
                    sentence_chunk_tokens = 0
                
                # 添加句子到当前束
                sentence_chunk.append(sentence)
                sentence_chunk_tokens += sentence_token_count
            
            # 保存最后一个句子束
            if sentence_chunk:
                chunks.append(' '.join(sentence_chunk))
        else:
            # 如果当前chunk加上新段落超过限制，保存当前chunk并开始新chunk
            if current_token_count + para_token_count > max_tokens and current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = []
                current_token_count = 0

            # 添加段落到当前chunk
            current_chunk.append(para)
            current_token_count += para_token_count

    # 添加最后一个chunk
    if current_chunk:
        chunks.append('\n\n'.join(current_chunk))

    return chunks



def fix_csv_columns(csv_content, expected_column_count):
    """修复CSV列数不匹配的问题"""
    try:
        lines = csv_content.strip().split('\n')
        fixed_lines = []
        
        for line in lines:
            reader = csv.reader([line])
            row = next(reader)
            
            # 检查列数
            if len(row) < expected_column_count:
                # 列数不足，补充空列
                row.extend([''] * (expected_column_count - len(row)))
            elif len(row) > expected_column_count:
                # 列数过多，截断
                row = row[:expected_column_count]
            
            # 重新构建CSV行
            output = io.StringIO()
            writer = csv.writer(output, quoting=csv.QUOTE_MINIMAL)  # 使用QUOTE_MINIMAL处理特殊字符
            writer.writerow(row)
            fixed_line = output.getvalue().strip()
            fixed_lines.append(fixed_line)
            output.close()
        
        return '\n'.join(fixed_lines)
    except Exception as e:
        logger.error(f"修复CSV列时出错: {e}")
        return None


def merge_csv_chunks_improved(chunks, original_header):
    """改进的CSV块合并函数，确保正确合并所有块并保留列结构"""
    if not chunks:
        return None
    
    try:
        # 解析第一个块获取翻译后的表头
        first_chunk = io.StringIO(chunks[0])
        reader = csv.reader(first_chunk)
        translated_header = next(reader)
        first_chunk_rows = list(reader)
        
        # 创建一个新的CSV用于合并结果
        output = io.StringIO()
        writer = csv.writer(output, quoting=csv.QUOTE_MINIMAL)  # 使用QUOTE_MINIMAL处理特殊字符
        writer.writerow(translated_header)
        writer.writerows(first_chunk_rows)
        
        # 合并其余块的数据行(跳过表头)
        for chunk in chunks[1:]:
            chunk_io = io.StringIO(chunk)
            reader = csv.reader(chunk_io)
            next(reader)  # 跳过表头
            writer.writerows(reader)
        
        result = output.getvalue()
        output.close()
        return result
    except Exception as e:
        logger.error(f"合并CSV块时出错: {e}")
        return chunks[0]  # 返回第一个块作为备用


def batch_translate_and_optimize(api_key, model, contents, target_language, seo_keywords=None,
                                 preserve_formatting=True, custom_instructions="", api_base=None):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """批量翻译和优化内容，带进度更新"""
    results = []
    total_files = len(contents)
    for i, content in enumerate(contents, 1):
        logger.info(f"处理进度: {i}/{total_files}")
        translated_content = translate_and_optimize(api_key, model, content, target_language, seo_keywords,
                                                    preserve_formatting, custom_instructions, api_base)
        results.append(translated_content)
    return results


def evaluate_translation_quality(original, translated):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """评估翻译质量，使用BLEU评分"""
    original_words = original.split()
    translated_words = translated.split()
    bleu_score = sentence_bleu([original_words], translated_words)
    return round(bleu_score * 100, 2)

def restore_section_headers(original_text, translated_text):
    """
    恢复翻译后文本中丢失的章节标题标记
    
    这个函数专门寻找原始文本中的Markdown标题（###、##等），
    然后在翻译文本中找到对应的翻译行，并恢复标题标记。
    """
    # 将文本分行
    original_lines = original_text.split('\n')
    translated_lines = translated_text.split('\n')
    
    # 找出原始文本中所有的标题行
    header_indexes = {}
    for i, line in enumerate(original_lines):
        match = re.match(r'^(#{1,6})\s+(.+)$', line)
        if match:
            level_marks = match.group(1)  # 标题级别（###、##等）
            header_text = match.group(2)  # 标题文本
            
            # 如果标题后面紧跟着图片行，记录这种特殊情况
            has_image_after = False
            if i+1 < len(original_lines) and original_lines[i+1].strip().startswith('!['):
                has_image_after = True
            
            header_indexes[i] = {
                'level': level_marks,
                'text': header_text,
                'has_image_after': has_image_after
            }
    
    # 修复翻译文本中丢失的标题
    fixed_translated_lines = []
    i = 0
    image_pattern = r'!\[.*?\]\(.*?\)'
    
    while i < len(translated_lines):
        line = translated_lines[i]
        
        # 如果已经是标题格式，保留原样
        if re.match(r'^#{1,6}\s+', line):
            fixed_translated_lines.append(line)
            i += 1
            continue
        
        # 检查当前行是否是图片行
        is_image_line = bool(re.match(image_pattern, line.strip()))
        
        # 如果当前行是图片行，检查是否应该在前面添加标题
        if is_image_line and i > 0:
            # 查找原始文本中图片行前面的标题行
            for orig_idx, header_info in header_indexes.items():
                if header_info['has_image_after']:
                    # 这里可以添加更复杂的启发式方法来匹配正确的标题
                    # 简单起见，我们假设翻译后文本保持了相似的结构
                    # 查找前面几行中可能对应的翻译标题
                    for back_idx in range(1, min(4, i+1)):
                        prev_line = translated_lines[i-back_idx].strip()
                        # 如果前面的行不是图片、不是空行，不是已有标题，可能是我们要找的标题
                        if (prev_line and 
                            not re.match(image_pattern, prev_line) and
                            not re.match(r'^#{1,6}\s+', prev_line) and
                            len(prev_line) < 100):  # 标题一般不会太长
                            
                            # 修复前一行，添加标题标记
                            fixed_translated_lines[-back_idx] = f"{header_info['level']} {prev_line}"
                            break
        
        fixed_translated_lines.append(line)
        i += 1
    
    return '\n'.join(fixed_translated_lines)

    
def translate_csv_improved(api_key, model, csv_content, target_language, original_header=None, api_base=None):
    time.sleep(2)  # 添加2秒延迟，避免频繁API调用
    """改进的CSV翻译函数，确保保留所有列名和处理包含逗号的字段"""
    logger.info(f"开始翻译CSV内容到{target_language}...")

    # 构建系统提示，明确强调对CSV格式的要求
    system_prompt = f"""你是一位专业的翻译专家。
    将以下CSV格式数据中的文本内容翻译成{target_language}。

    要求:
    - 第一行是CSV的列名，必须完整翻译所有列名，不可省略任何列名
    - 必须保留CSV的所有列结构，确保所有列均被保留，不可合并或删除任何列
    - 重要：单元格内容中可能包含逗号，这些逗号不是列分隔符，翻译时必须保持在单元格内部
    - 非常重要：你收到的CSV已经正确解析，每一行的字段数必须保持不变
    - 只翻译文本内容，保留CSV的结构和格式
    - 不要翻译数字、日期、邮箱、URL和代码
    - 专业术语要准确翻译
    - 返回完整的CSV格式数据，保持所有列的完整性
    """
    # 创建客户端
    client = SimpleOpenAIClient(api_key, api_base)

    # API调用及错误处理和重试
    max_retries = 15  # 增加重试次数
    retry_delay = 10

    for attempt in range(max_retries):
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": csv_content}
            ]

            response = client.create_chat_completion(model, messages, temperature=0.3)
            translated_content = response["choices"][0]["message"]["content"].strip()
            
            # 验证翻译结果是否保留了所有列
            if original_header:
                try:
                    # 解析翻译后的CSV内容
                    translated_csv = io.StringIO(translated_content)
                    reader = csv.reader(translated_csv)
                    translated_header = next(reader)
                    
                    # 检查列数是否匹配
                    if len(translated_header) != len(original_header):
                        logger.warning(f"翻译后的列数({len(translated_header)})与原始列数({len(original_header)})不匹配")
                        logger.warning(f"原始列: {original_header}")
                        logger.warning(f"翻译后列: {translated_header}")
                        
                        # 如果列数不匹配，尝试修复
                        fixed_content = fix_csv_columns(translated_content, len(original_header))
                        if fixed_content:
                            translated_content = fixed_content
                except Exception as e:
                    logger.warning(f"验证CSV列时出错: {e}")
            
            logger.info("CSV翻译完成。")
            return translated_content
        except Exception as e:
            if "context_length_exceeded" in str(e):
                logger.warning("CSV内容过长，需要拆分处理")
                raise ValueError("Content too long for model context")
            elif "rate_limit" in str(e).lower() and attempt < max_retries - 1:
                logger.warning(f"遇到速率限制，{retry_delay}秒后重试...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 指数退避
            else:
                logger.error(f"API 请求失败，重试 {attempt + 1}/{max_retries}: {e}")
                time.sleep(retry_delay)

    logger.error("API 请求失败，达到最大重试次数。")
    return None

def process_csv_file_improved(api_key, model, file_path, output_path, target_language, api_base=None):
    time.sleep(3)  # 3秒延迟
    """改进的CSV文件处理函数，确保保留列结构"""
    logger.info(f"处理CSV文件: {file_path}")

    try:
        # 读取CSV文件内容（使用csv模块正确处理）
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                # 使用csv模块读取，而不是直接读取文本
                reader = csv.reader(file)
                all_rows = list(reader)
                original_header = all_rows[0]
                data_rows = all_rows[1:]
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding='gb18030') as file:
                reader = csv.reader(file)
                all_rows = list(reader)
                original_header = all_rows[0]
                data_rows = all_rows[1:]

        # 将CSV数据转换为纯文本进行翻译，但保留正确的CSV结构
        csv_buffer = io.StringIO()
        writer = csv.writer(csv_buffer, quoting=csv.QUOTE_ALL)  # 使用QUOTE_ALL确保所有字段被引号包围
        writer.writerow(original_header)
        writer.writerows(data_rows)
        content = csv_buffer.getvalue()
        csv_buffer.close()

        # 估算文件大小和token数量
        estimated_tokens = count_tokens(content)
        logger.info(f"CSV文件估计有 {estimated_tokens} tokens")

        # 如果文件过大，直接拆分处理
        if estimated_tokens > 6000:  # 设置一个安全阈值
            logger.info("CSV内容较大，直接拆分处理")
            chunks, header = split_csv_improved(content, max_rows=5)  # 将每块的行数从10减少到5

            if not chunks:
                logger.error("CSV拆分失败，无法处理")
                return False

            logger.info(f"CSV拆分为{len(chunks)}个块进行处理")
            translated_chunks = []

            # 翻译每个块
            for i, chunk in enumerate(chunks):
                logger.info(f"处理CSV块 {i + 1}/{len(chunks)}")
                max_retries = 5
                retry_delay = 10
                success = False

                for attempt in range(max_retries):
                    try:
                        translated_chunk = translate_csv_improved(api_key, model, chunk, target_language, original_header, api_base)
                        translated_chunks.append(translated_chunk)
                        success = True
                        break
                    except Exception as chunk_e:
                        if "rate_limit" in str(chunk_e).lower():
                            wait_time = retry_delay * (2 ** attempt)
                            logger.warning(f"速率限制，等待{wait_time}秒后重试... ({attempt + 1}/{max_retries})")
                            time.sleep(wait_time)
                        elif "context_length_exceeded" in str(chunk_e):
                            # 如果还是太大，进一步拆分
                            logger.warning(f"块 {i + 1} 仍然太大，需要进一步拆分")
                            sub_chunks, _ = split_csv_improved(chunk, max_rows=2)  # 进一步减小块大小
                            sub_translated = []

                            for j, sub_chunk in enumerate(sub_chunks):
                                logger.info(f"处理子块 {j + 1}/{len(sub_chunks)} (来自块 {i + 1})")
                                for sub_attempt in range(max_retries):
                                    try:
                                        sub_translated_chunk = translate_csv_improved(api_key, model, sub_chunk, target_language,
                                                                              original_header, api_base)
                                        sub_translated.append(sub_translated_chunk)
                                        break
                                    except Exception as sub_e:
                                        if "rate_limit" in str(sub_e).lower() and sub_attempt < max_retries - 1:
                                            sub_wait = retry_delay * (2 ** sub_attempt)
                                            logger.warning(f"子块速率限制，等待{sub_wait}秒后重试...")
                                            time.sleep(sub_wait)
                                        else:
                                            logger.error(f"处理子块 {j + 1} 失败: {sub_e}")
                                            return False

                            # 合并子块
                            merged_sub = merge_csv_chunks_improved(sub_translated, original_header)
                            translated_chunks.append(merged_sub)
                            success = True
                            break
                        else:
                            logger.error(f"处理块 {i + 1} 错误: {chunk_e}")
                            if attempt == max_retries - 1:
                                return False
                            time.sleep(retry_delay)

                if not success:
                    logger.error(f"处理块 {i + 1} 达到最大重试次数，失败")
                    return False

                # 每个块处理后增加延迟以避免速率限制
                time.sleep(10)  # 从5秒增加到10秒

            # 合并翻译结果
            if translated_chunks:
                merged_content = merge_csv_chunks_improved(translated_chunks, original_header)

                # 保存合并后的翻译结果 - 确保正确处理CSV结构
                # 首先解析合并后的内容
                merged_csv = io.StringIO(merged_content)
                reader = csv.reader(merged_csv)
                merged_rows = list(reader)
                
                # 然后使用CSV模块正确写入，确保字段被引号包围
                with open(output_path, 'w', encoding='utf-8', newline='') as file:
                    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
                    writer.writerows(merged_rows)

                logger.info(f"CSV拆分翻译合并完成: {output_path}")
                return True
            else:
                logger.error("没有成功翻译的块")
                return False
        else:
            # 文件较小，尝试直接翻译
            logger.info("尝试直接翻译整个CSV")
            try:
                translated_content = translate_csv_improved(api_key, model, content, target_language, original_header, api_base)

                # 保存翻译结果 - 确保正确处理CSV结构
                # 首先解析翻译后的内容
                translated_csv = io.StringIO(translated_content)
                reader = csv.reader(translated_csv)
                translated_rows = list(reader)
                
                # 然后使用CSV模块正确写入，确保字段被引号包围
                with open(output_path, 'w', encoding='utf-8', newline='') as file:
                    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
                    writer.writerows(translated_rows)

                logger.info(f"CSV翻译完成: {output_path}")
                return True
            except ValueError as e:
                if "Content too long" in str(e):
                    # 如果整体翻译失败，返回尝试拆分处理
                    logger.info("直接翻译失败，回退到拆分处理模式")
                    return process_csv_file_improved(api_key, model, file_path, output_path, target_language, api_base)
                else:
                    raise
    except Exception as e:
        logger.error(f"处理CSV文件错误: {e}")
        return False

def preserve_document_structure(original_content, translated_content):
    """
    保留原始文档的结构和格式，特别是Markdown标题和列表结构
    """
    # 首先提取原始内容中的所有标题行
    header_pattern = r'(#{1,6}\s+[^\n]+)'
    original_headers = re.findall(header_pattern, original_content)
    
    # 创建结构化后的内容
    structured_content = translated_content
    
    # 确保原始标题结构被保留
    for i, header in enumerate(original_headers):
        # 获取标题级别(#的数量)
        level_match = re.match(r'(#+)', header)
        if not level_match:
            continue
            
        level = level_match.group(1)
        
        # 在翻译内容中寻找对应的部分
        # 这里采用一个简单的策略：查找内容中第i个标题，不管它的文本如何
        translated_headers = re.findall(header_pattern, structured_content)
        
        if i < len(translated_headers):
            # 替换为保持原始格式的标题(保留#的数量)
            # 但使用翻译后的文本内容
            translated_text = re.sub(r'^#+\s+', '', translated_headers[i]).strip()
            new_header = f"{level} {translated_text}"
            structured_content = structured_content.replace(translated_headers[i], new_header)
    
    # 保留列表格式
    # 查找所有列表项(- 或* 开头的行)
    list_pattern = r'^(\s*[-*]\s+[^\n]+)$'
    original_list_items = re.findall(list_pattern, original_content, re.MULTILINE)
    translated_list_items = re.findall(list_pattern, structured_content, re.MULTILINE)
    
    # 确保列表格式一致
    if len(original_list_items) == len(translated_list_items):
        for i, (orig_item, trans_item) in enumerate(zip(original_list_items, translated_list_items)):
            # 提取列表符号和缩进
            orig_prefix = re.match(r'(\s*[-*]\s+)', orig_item)
            if orig_prefix:
                prefix = orig_prefix.group(1)
                # 替换翻译项的前缀，确保一致性
                trans_text = re.sub(r'^\s*[-*]\s+', '', trans_item)
                new_item = f"{prefix}{trans_text}"
                structured_content = structured_content.replace(trans_item, new_item)
    
    return structured_content   

def sanitize_filename(filename):
    """移除文件名中的非法字符，避免创建目录时报错"""
    return re.sub(r'[<>:"/\\|?*]', '_', filename)
def process_directory(api_key, model, input_dir, output_dir, target_language, file_types, seo_optimize,
                      preserve_formatting, custom_instructions, api_base=None):
    """递归处理目录下的所有匹配文件，保持原始目录结构"""

    file_types = file_types.split(',')
    for i in range(len(file_types)):
        if not file_types[i].startswith('.'):
            file_types[i] = '.' + file_types[i]  # 确保文件类型有 `.`

    input_files = []

    # **📌 遍历所有子目录，收集需要处理的文件**
    for root, _, files in os.walk(input_dir):
        for file in files:
            if any(file.endswith(ft) for ft in file_types):
                input_files.append(os.path.join(root, file))

    if not input_files:
        print(f"❌ 没有找到 {file_types} 类型的文件。")
        return

    print(f"✅ 找到 {len(input_files)} 个文件需要翻译...")

    # **📌 处理文件，并显示进度**
    success_count = 0
    for i, filepath in enumerate(input_files, 1):
        # 计算相对路径，保持完整的目录结构
        rel_path = os.path.relpath(filepath, input_dir)
        # 构建输出文件路径，保持相同的目录结构
        file_name = os.path.basename(rel_path)
        file_base, file_ext = os.path.splitext(file_name)
        output_file_name = f"{file_base}_{target_language.lower()}{file_ext}"
        
        # 获取输出文件的目录部分
        output_subdir = os.path.join(output_dir, os.path.dirname(rel_path))
        # 确保输出子目录存在
        os.makedirs(output_subdir, exist_ok=True)
        
        # 完整的输出文件路径
        output_path = os.path.join(output_subdir, output_file_name)

        print(f"📄 正在处理 ({i}/{len(input_files)}): {filepath}")
        print(f"   输出到: {output_path}")
        
        # 直接调用处理函数，传递完整的输出路径
        if filepath.lower().endswith('.csv'):
            success = process_csv_file_improved(api_key, model, filepath, output_path, target_language, api_base)
        else:
            success = process_text_file(api_key, model, filepath, output_path, target_language,
                                        seo_optimize, preserve_formatting, custom_instructions, api_base)
        
        if success:
            success_count += 1
            print(f"✅ 文件处理成功: {output_path}")
        else:
            print(f"❌ 文件处理失败: {filepath}")

    print(f"🎉 处理完成! {success_count}/{len(input_files)} 个文件成功翻译。")  

## 翻译主函数

In [ ]:
import os
# 测试代码
if __name__ == "__main__":
    # 测试参数
    API_KEY = "xxxx"  # 替换为你的DeepSeek API密钥
    MODEL = "gpt-3.5-turbo"  # 或DeepSeek提供的具体模型名称
    TARGET_LANGUAGE = "en"  # 翻译语言选择-此处设置为英语
    # 其余参数保持不变
    SEO_KEYWORDS = ["laser", "optical lens", "diffraction", "refractive index", "grating"]
    INPUT_DIR = r'./拆分结果'  # 输入文件夹路径
    OUTPUT_DIR = r'./翻译结果'  # 输出文件夹路径
    FILE_TYPES = ".txt,.md,.csv"  # 要翻译的文件类型
    
    # 可能需要更改的API基础URL
    API_BASE = "https://api.openai.com/v1"  # DeepSeek的API基础URL
    
    print("📋 开始翻译系统")
    print(f"🔮 目标语言: {TARGET_LANGUAGE}")
    print(f"📁 处理目录: {INPUT_DIR}")
    
    try:
        # 批量目录翻译
        process_directory(API_KEY, MODEL, INPUT_DIR, OUTPUT_DIR, TARGET_LANGUAGE,
                         FILE_TYPES, seo_optimize=True, preserve_formatting=True, 
                         custom_instructions="", api_base=API_BASE)
    except Exception as e:
        print(f"❌ 处理过程中出现错误: {e}")
        import traceback
        print(traceback.format_exc())  # 打印详细错误堆栈
        
        # 尝试重新处理失败的文件，使用更保守的设置
        print("\n🔄 尝试使用更保守的设置重新处理未成功的文件...")
        
        # 获取所有匹配的文件
        input_files = []
        file_type_list = FILE_TYPES.split(',')
        for i in range(len(file_type_list)):
            if not file_type_list[i].startswith('.'):
                file_type_list[i] = '.' + file_type_list[i]
                
        for root, _, files in os.walk(INPUT_DIR):
            for file in files:
                if any(file.endswith(ft) for ft in file_type_list):
                    input_files.append(os.path.join(root, file))
        
        # 使用更保守的设置处理所有未处理成功的文件
        for filepath in input_files:
            # 检查对应的输出文件是否已存在
            rel_path = os.path.relpath(filepath, INPUT_DIR)
            file_name = os.path.basename(rel_path)
            file_base, file_ext = os.path.splitext(file_name)
            output_file_name = f"{file_base}_{TARGET_LANGUAGE.lower()}{file_ext}"
            output_subdir = os.path.join(OUTPUT_DIR, os.path.dirname(rel_path))
            output_path = os.path.join(output_subdir, output_file_name)
            
            # 如果输出文件不存在，使用更保守的设置处理
            if not os.path.exists(output_path):
                print(f"🔄 重新处理未成功的文件: {filepath}")
                os.makedirs(output_subdir, exist_ok=True)
                
                # 对于CSV文件和普通文本文件使用不同的处理函数
                if filepath.lower().endswith('.csv'):
                    process_csv_file_improved(API_KEY, MODEL, filepath, output_path, TARGET_LANGUAGE, api_base=None)
                else:
                    # 使用更保守的SEO设置和更小的块大小
                    process_text_file(API_KEY, MODEL, filepath, output_path, TARGET_LANGUAGE,
                                     seo_optimize=False, preserve_formatting=True, 
                                     custom_instructions="请使用更保守的翻译策略，优先保证准确性", api_base=None)
        
        print("🎉 重新处理完成!")

📋 开始翻译系统
🔮 目标语言: en
📁 处理目录: ./拆分结果
✅ 找到 1540 个文件需要翻译...
📄 正在处理 (1/1540): ./拆分结果\CSV\25 mm行程手动线性位移台_产品列表.csv
   输出到: ./翻译结果\CSV\25 mm行程手动线性位移台_产品列表_en.csv


2025-03-27 12:54:01,429 - INFO - 处理CSV文件: ./拆分结果\CSV\25 mm行程手动线性位移台_产品列表.csv
2025-03-27 12:54:01,434 - INFO - CSV文件估计有 521 tokens
2025-03-27 12:54:01,435 - INFO - 尝试直接翻译整个CSV
2025-03-27 12:54:03,437 - INFO - 开始翻译CSV内容到en...
2025-03-27 12:54:09,783 - INFO - CSV翻译完成。
2025-03-27 12:54:09,785 - INFO - CSV翻译完成: ./翻译结果\CSV\25 mm行程手动线性位移台_产品列表_en.csv


✅ 文件处理成功: ./翻译结果\CSV\25 mm行程手动线性位移台_产品列表_en.csv
📄 正在处理 (2/1540): ./拆分结果\CSV\30 mm同轴分束立方安装座_产品列表.csv
   输出到: ./翻译结果\CSV\30 mm同轴分束立方安装座_产品列表_en.csv


2025-03-27 12:54:12,788 - INFO - 处理CSV文件: ./拆分结果\CSV\30 mm同轴分束立方安装座_产品列表.csv
2025-03-27 12:54:12,790 - INFO - CSV文件估计有 689 tokens
2025-03-27 12:54:12,791 - INFO - 尝试直接翻译整个CSV
2025-03-27 12:54:14,797 - INFO - 开始翻译CSV内容到en...
2025-03-27 12:54:22,524 - INFO - CSV翻译完成。
2025-03-27 12:54:22,527 - INFO - CSV翻译完成: ./翻译结果\CSV\30 mm同轴分束立方安装座_产品列表_en.csv


✅ 文件处理成功: ./翻译结果\CSV\30 mm同轴分束立方安装座_产品列表_en.csv
📄 正在处理 (3/1540): ./拆分结果\CSV\30 mm同轴安装板_产品列表.csv
   输出到: ./翻译结果\CSV\30 mm同轴安装板_产品列表_en.csv


...

✅ 文件处理成功: ./翻译结果\落地页\钝化不锈钢面包板_落地页描述_en.txt
📄 正在处理 (1539/1540): ./拆分结果\落地页\高分辨率光纤光谱仪_落地页描述.txt
   输出到: ./翻译结果\落地页\高分辨率光纤光谱仪_落地页描述_en.txt


2025-03-28 00:14:03,139 - INFO - 处理文本文件: ./拆分结果\落地页\高分辨率光纤光谱仪_落地页描述.txt
2025-03-28 00:14:05,145 - INFO - 开始提取SEO关键词...
2025-03-28 00:14:06,667 - INFO - 成功提取关键词：6个
2025-03-28 00:14:06,668 - INFO - 成功提取关键词: {'primary_keywords': ['高分辨率光纤光谱仪', '光谱仪', '高分辨率光谱'], 'long_tail_keywords': ['深紫外波段光谱探测', 'LBS7500光谱仪', '多通道光谱拼接']}
2025-03-28 00:14:06,670 - INFO - 文件总内容估计有 262 tokens
2025-03-28 00:14:06,672 - INFO - 文件较小，尝试直接翻译
2025-03-28 00:14:08,672 - INFO - 开始翻译内容到en...
2025-03-28 00:14:08,674 - INFO - 估计内容有 262 tokens
2025-03-28 00:14:08,675 - INFO - 设置完成tokens: 4096
2025-03-28 00:14:08,676 - INFO - 使用SEO关键词优化：高分辨率光纤光谱仪, 光谱仪, 高分辨率光谱, 深紫外波段光谱探测, LBS7500光谱仪
2025-03-28 00:14:12,604 - INFO - 翻译完成。
2025-03-28 00:14:12,606 - INFO - 文件翻译完成: ./翻译结果\落地页\高分辨率光纤光谱仪_落地页描述_en.txt


✅ 文件处理成功: ./翻译结果\落地页\高分辨率光纤光谱仪_落地页描述_en.txt
📄 正在处理 (1540/1540): ./拆分结果\落地页\高性能带通滤光片_落地页描述.txt
   输出到: ./翻译结果\落地页\高性能带通滤光片_落地页描述_en.txt


2025-03-28 00:14:14,608 - INFO - 处理文本文件: ./拆分结果\落地页\高性能带通滤光片_落地页描述.txt
2025-03-28 00:14:16,615 - INFO - 开始提取SEO关键词...
2025-03-28 00:14:18,132 - INFO - 成功提取关键词：8个
2025-03-28 00:14:18,132 - INFO - 成功提取关键词: {'primary_keywords': ['高性能带通滤光片', '滤光片', '光学元件'], 'long_tail_keywords': ['带机械卡环', '中心波长', '阻带光密度', '磁控溅射镀膜工艺', '荧光成像']}
2025-03-28 00:14:18,133 - INFO - 文件总内容估计有 302 tokens
2025-03-28 00:14:18,134 - INFO - 文件较小，尝试直接翻译
2025-03-28 00:14:20,136 - INFO - 开始翻译内容到en...
2025-03-28 00:14:20,137 - INFO - 估计内容有 302 tokens
2025-03-28 00:14:20,138 - INFO - 设置完成tokens: 4096
2025-03-28 00:14:20,139 - INFO - 使用SEO关键词优化：高性能带通滤光片, 滤光片, 光学元件, 带机械卡环, 中心波长
2025-03-28 00:14:24,520 - INFO - 翻译完成。
2025-03-28 00:14:24,522 - INFO - 文件翻译完成: ./翻译结果\落地页\高性能带通滤光片_落地页描述_en.txt


✅ 文件处理成功: ./翻译结果\落地页\高性能带通滤光片_落地页描述_en.txt
🎉 处理完成! 1538/1540 个文件成功翻译。


## 合并
#### 成功合并后会将通用内容文件移动到其它文件夹存档，产品详情文件是原地合并

In [1]:
import os
import glob
import re
import datetime
import shutil
from pathlib import Path

def merge_common_content_to_product_details(directory, log_entries, archive_dir):
    """
    将通用内容文件合并到每个产品详情文件的末尾，然后将通用内容文件移动到指定文件夹
    成功合并后还会删除备份文件
    
    Args:
        directory: 包含产品详情文件和通用内容文件的目录
        log_entries: 用于记录日志的列表
        archive_dir: 用于存放通用内容文件的存档目录
    """
    log_entries.append(f"开始处理目录: {directory}")
    
    # 查找所有通用内容文件
    common_files = glob.glob(os.path.join(directory, "*_通用内容*.txt"))
    
    if not common_files:
        log_entries.append(f"在 {directory} 中未找到通用内容文件")
        return
    
    # 用于跟踪已处理的文件
    processed_files = []
    has_errors = False
    
    # 对每个通用内容文件进行处理
    for common_file in common_files:
        try:
            # 读取通用内容文件
            with open(common_file, 'r', encoding='utf-8') as f:
                common_content = f.read()
            
            # 提取基础文件名（不含"_通用内容"部分）
            base_name = os.path.basename(common_file).replace("_通用内容.txt", "").replace("_通用内容_en.txt", "")
            
            # 查找所有匹配的产品详情文件
            product_pattern = f"{base_name}_*.txt"
            product_files = glob.glob(os.path.join(directory, product_pattern))
            
            # 过滤掉通用内容文件本身以及可能存在的备份文件
            product_files = [f for f in product_files if not re.search(r"_通用内容(_[a-zA-Z\d]+)?\.txt$", f) 
                             and not f.endswith(".bak")]
            
            log_entries.append(f"在 {directory} 中找到 {len(product_files)} 个匹配的产品详情文件")
            
            # 存储创建的备份文件列表
            backup_files = []
            file_errors = []
            
            # 将通用内容合并到每个产品详情文件
            for product_file in product_files:
                # 检查此文件是否已经处理过
                if product_file in processed_files:
                    continue
                
                try:
                    # 读取产品文件内容
                    with open(product_file, 'r', encoding='utf-8') as f:
                        product_content = f.read()
                    
                    # 创建备份文件
                    backup_file = f"{product_file}.bak"
                    with open(backup_file, 'w', encoding='utf-8') as f:
                        f.write(product_content)
                    
                    # 添加备份文件到列表
                    backup_files.append(backup_file)
                    
                    # 添加通用内容到产品详情文件
                    with open(product_file, 'a', encoding='utf-8') as f:
                        f.write("\n\n")
                        f.write(common_content)
                    
                    # 将文件添加到已处理列表
                    processed_files.append(product_file)
                
                except Exception as e:
                    has_errors = True
                    file_errors.append(f"处理文件 {os.path.basename(product_file)} 时出错: {str(e)}")
            
            # 如果有文件处理出错，记录详细信息
            if file_errors:
                for error in file_errors:
                    log_entries.append(error)
                log_entries.append(f"由于部分合并失败，保留通用内容文件和备份文件: {os.path.basename(common_file)}")
            else:
                # 删除备份文件
                for backup_file in backup_files:
                    try:
                        os.remove(backup_file)
                    except Exception as e:
                        has_errors = True
                        log_entries.append(f"删除备份文件 {os.path.basename(backup_file)} 时出错: {str(e)}")
                
                # 移动通用内容文件而不是删除它
                try:
                    common_filename = os.path.basename(common_file)
                    dest_path = os.path.join(archive_dir, common_filename)
                    
                    # 移动文件（如果存在同名文件则直接覆盖）
                    shutil.move(common_file, dest_path)
                    log_entries.append(f"已将通用内容文件移动到: {dest_path}")
                except Exception as e:
                    has_errors = True
                    log_entries.append(f"移动通用内容文件 {os.path.basename(common_file)} 时出错: {str(e)}")
        
        except Exception as e:
            has_errors = True
            log_entries.append(f"处理通用内容文件 {os.path.basename(common_file)} 时出错: {str(e)}")
    
    if not has_errors:
        log_entries.append(f"目录 {directory} 处理完成，共处理 {len(processed_files)} 个文件")
        print(f"目录 {directory} 处理完成，共处理 {len(processed_files)} 个文件")
    
    return has_errors

def process_all_directories(base_dir, archive_dir=None):
    """
    处理基础目录下的所有产品系列目录
    
    Args:
        base_dir: 包含多个产品系列目录的基础目录
        archive_dir: 用于存放通用内容文件的存档目录，如果为None则使用默认路径
    """
    # 确保日志目录存在
    log_dir = os.path.join(os.path.dirname(os.path.dirname(base_dir)), "日志")
    os.makedirs(log_dir, exist_ok=True)
    
    # 如果未指定存档目录，则使用默认路径
    if archive_dir is None:
        archive_dir = os.path.join(os.path.dirname(os.path.dirname(base_dir)), "通用内容存档")
    
    # 确保存档目录存在
    os.makedirs(archive_dir, exist_ok=True)
    
    # 创建日志文件
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file_path = os.path.join(log_dir, f"合并日志_{timestamp}.txt")
    
    # 初始化日志条目
    log_entries = [
        f"合并处理开始时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        f"基础目录: {base_dir}",
        f"通用内容文件存档目录: {archive_dir}"
    ]
    
    # 查找所有子目录
    subdirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    
    if not subdirs:
        log_entries.append(f"在 {base_dir} 中未找到子目录")
        # 写入日志
        with open(log_file_path, 'w', encoding='utf-8') as log_file:
            for entry in log_entries:
                log_file.write(f"{entry}\n")
        print(f"未找到子目录。详细日志请查看: {log_file_path}")
        return
    
    log_entries.append(f"找到 {len(subdirs)} 个子目录可处理")
    print(f"找到 {len(subdirs)} 个子目录可处理")
    
    # 处理每个子目录
    has_any_error = False
    for subdir in subdirs:
        full_path = os.path.join(base_dir, subdir)
        has_error = merge_common_content_to_product_details(full_path, log_entries, archive_dir)
        if has_error:
            has_any_error = True
    
    log_entries.append(f"合并处理结束时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 写入日志
    with open(log_file_path, 'w', encoding='utf-8') as log_file:
        for entry in log_entries:
            log_file.write(f"{entry}\n")
    
    status = "有错误发生" if has_any_error else "全部成功"
    print(f"合并处理完成，状态: {status}! 详细日志请查看: {log_file_path}")

# 用法示例
if __name__ == "__main__":
    # 设置基础目录 - 这应该是包含产品详情目录的路径
    # 例如，如果你的结构是 ./拆分结果/产品详情/聚合物涡旋波片/...
    # 那么这里应该设置为 "./拆分结果/产品详情"
    base_directory = r'./翻译结果/产品详情'  # 修改为你的路径
    
    # 设置通用内容文件的存档目录（可选）
    # 如果不设置，默认为 "../通用内容存档"（相对于base_directory的上两级目录）
    archive_directory = r'./通用内容存档'  # 修改为你想要的输出路径
    
    # 处理所有产品系列目录
    process_all_directories(base_directory, archive_directory)

找到 100 个子目录可处理
目录 ./翻译结果/产品详情\25 mm行程手动线性位移台 处理完成，共处理 11 个文件
目录 ./翻译结果/产品详情\30 mm同轴分束立方安装座 处理完成，共处理 5 个文件
目录 ./翻译结果/产品详情\30 mm同轴安装板 处理完成，共处理 11 个文件
目录 ./翻译结果/产品详情\30 mm同轴平片光学元件安装座 处理完成，共处理 4 个文件
目录 ./翻译结果/产品详情\30 mm同轴立方体 处理完成，共处理 10 个文件
目录 ./翻译结果/产品详情\30 mm同轴立方体配件 处理完成，共处理 9 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统45度安装调整架 处理完成，共处理 13 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统XYZ位移调整架 处理完成，共处理 2 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统XY位移调整架 处理完成，共处理 7 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统反射镜架 处理完成，共处理 3 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统对准板 处理完成，共处理 3 个文件
目录 ./翻译结果/产品详情\30 mm同轴系统旋转调整架 处理完成，共处理 4 个文件
目录 ./翻译结果/产品详情\45度安装调整架 处理完成，共处理 20 个文件
目录 ./翻译结果/产品详情\60 mm同轴安装板 处理完成，共处理 1 个文件
目录 ./翻译结果/产品详情\60 mm同轴系统反射镜架 处理完成，共处理 2 个文件
目录 ./翻译结果/产品详情\60 mm同轴系统旋转调整架 处理完成，共处理 2 个文件
目录 ./翻译结果/产品详情\OAS光学分析软件 处理完成，共处理 1 个文件
目录 ./翻译结果/产品详情\SM05透镜套筒 处理完成，共处理 12 个文件
目录 ./翻译结果/产品详情\SM1透镜套筒 处理完成，共处理 18 个文件
目录 ./翻译结果/产品详情\SM2透镜套筒 处理完成，共处理 12 个文件
目录 ./翻译结果/产品详情\Ø12.7 mm不锈钢光学接杆 处理完成，共处理 16 个文件
目录 ./翻译结果/产品详情\Ø12.7 mm接杆套环 处理完成，共处理 9 个文件
目录 ./翻译结果/产品详情\Ø12.7 mm接杆支